# Homework №1 - Data collecting and cleansing

## Data preparing

First of all let's import all the libraries we need

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import requests
import time
from typing import List
from tqdm import tqdm
from tqdm import tqdm_notebook

In [3]:
data = pd.read_csv('./data_2.csv', sep=',')

In [4]:
data

,DOI,Date,Journal,Title,Name,measurement_error,measurement_wavelength,measurement_method,normalised_name,raw_value,specifier
0,10.1016/j.optlaseng.2008.09.005,1/14/2009,Optics and Lasers in Engineering,AHIGHPRECISIONMEASUREMENTTECHNIQUEFOREVALUATIN...,Alcohol,0.00000,NaN,el_mylogic,CCO,1.34749,Refractive index (n)
1,10.1016/j.jiec.2018.12.038,12/29/2018,Journal of Industrial and Engineering Chemistry,IMPROVINGUNDERSTANDINGSOLVENTEFFECTSINTERMOLEC...,aceticacid,0.00000,NaN,el_mylogic,CC(O)=O,1.7,n
2,10.1016/j.jlumin.2015.02.010,2/14/2015,Journal of Luminescence,SPECTROSCOPICPROPERTIESTELLURITEGLASSESCODOPED...,Phosphate,0.00000,600 nm,el_mylogic,[O-][P]([O-])([O-])=O,1.54,Refractive index (~600 nm)
3,10.1039/B717069F_© The Royal Society of Chemis...,NaN,NaN,NaN,BK7,0.00000,NaN,snowball,NaN,1.518,refractive index
4,10.1529/biophysj.106.094946,10/23/2008,Biophysical Journal,QUANTITATIVECHARACTERIZATIONBIOLOGICALLIQUIDSF...,ethanol,0.00126,NaN,el_cde_text,CCO,1.35434 ± 0.00126,refractive index
...,...,...,...,...,...,...,...,...,...,...,...
4995,10.1016/j.sse.2007.06.004,7/30/2007,Solid-State Electronics,RADIATIVELINEWIDTHASINGLEIMPURITYMOLECULEINABI...,Biphenyl,0.00000,NaN,el_cde_tables,c1ccc(cc1)c2ccccc2,1.68,Refractive indices
4996,10.1039/B822982A,7/21/2009,Lab on a Chip,Tunable Liquid Gradient Refractive Index (L-GR...,CaCl2,0.00000,NaN,rsc_cde_text,"[['Ca', 1.0], ['Cl', 2.0]]",∼1.41,nD
4997,10.1078/0030-4026-00175,11/5/2004,Optik,OPTIMIZATIONCONDUCTINGPOLYMERTHINFILMOPTICALEL...,He-Ne,0.00000,NaN,el_cde_text,"[['He-', 1.0], ['Ne', 1.0]]",1.717,refractive index
4998,10.1016/j.optmat.2018.06.013,6/18/2018,Optical Materials,ENHANCEDPHOTOELECTRICPERFORMANCENANORODTIO2FIL...,ITO,0.00000,NaN,el_cde_text,"[['IT', 1.0], ['O', 1.0]]",1.92,refractive index


Let's check how many gaps we have in the lines, try to fill them

In [5]:
columns = list(data.columns)

In [6]:
columns

['DOI',
 'Date',
 'Journal',
 'Title',
 'Name',
 'measurement_error',
 'measurement_wavelength',
 'measurement_method',
 'normalised_name',
 'raw_value',
 'specifier']

In the inirial dataframe we have several colums leasted above.
Let's determine what value each parameter has:

1) **DOI (Digital Object Identifier)** - is a string of numbers, letters and symbols used to uniquely identify an article or document, and to provide it with a permanent web address (URL);

2) **Date** - date  of article publication;
3) **Journal** - the name of the journal in which the article was published;

4) **Title** - the title of the article;

5) **Name** - the systematic name, trivial name formula of the chemical compound;

6) **measurement_error** -  is the difference between a measured quantity and its true value;

7) **measurement_wavelength** -  important factor in the determination of refractive index using a spectrophotometer or other optical instrument;

8) **measurement_method** - procedures or techniques used to make these assignments and obtain the numerical or symbolic representation of the properties or characteristics being measured;

9) **normalised_name (SMILES)** - standard notation used in chemistry to represent the structure of molecules and chemical reactions using a short, linear string of characters;

10) **raw_value (reflecting index)** - the measure of bending of a light ray when passing from one medium to another during the experiment;

11) **specifier** - the type of raw_value value.

In [7]:
print("Missing values distribution by column: ")
print(data.isnull().mean())
print("")

Missing values distribution by column: 
DOI                       0.0000
Date                      0.0842
Journal                   0.0842
Title                     0.0842
Name                      0.0010
measurement_error         0.0000
measurement_wavelength    0.8858
measurement_method        0.0000
normalised_name           0.4116
raw_value                 0.0000
specifier                 0.0000
dtype: float64



Now we can easily look at the distribution of missing values in the given dataset, for example: in the column 'measurement_wavelength' 88,58% of data is missing, in 'normalised_name' - 41,16%, which is sucks actually:( Good news: in columns 'DOI', 'measurement_error', 'measurement_method', 'raw_value' and 'specifier' there are no missing values at all! Cool!

In [8]:
print("Column datatypes: ")
print(data.dtypes)

Column datatypes: 
DOI                        object
Date                       object
Journal                    object
Title                      object
Name                       object
measurement_error         float64
measurement_wavelength     object
measurement_method         object
normalised_name            object
raw_value                  object
specifier                  object
dtype: object


Now let's see at the type of data we have. All the columns have object as their datatype aside from 'measurement_error'. In pandas, object means either string or mixed type (numerical and non-numerical type mixed).

Finally, let’s make sure we remove any trailing characters and whitespace using 'strip':

In [9]:
str_cols = list(data.columns)
str_cols.remove('measurement_error')

In [10]:
for i in str_cols:
    data[i] = data[i].str.strip()

In [11]:
data.head()

,DOI,Date,Journal,Title,Name,measurement_error,measurement_wavelength,measurement_method,normalised_name,raw_value,specifier
0,10.1016/j.optlaseng.2008.09.005,1/14/2009,Optics and Lasers in Engineering,AHIGHPRECISIONMEASUREMENTTECHNIQUEFOREVALUATIN...,Alcohol,0.00000,NaN,el_mylogic,CCO,1.34749,Refractive index (n)
1,10.1016/j.jiec.2018.12.038,12/29/2018,Journal of Industrial and Engineering Chemistry,IMPROVINGUNDERSTANDINGSOLVENTEFFECTSINTERMOLEC...,aceticacid,0.00000,NaN,el_mylogic,CC(O)=O,1.7,n
2,10.1016/j.jlumin.2015.02.010,2/14/2015,Journal of Luminescence,SPECTROSCOPICPROPERTIESTELLURITEGLASSESCODOPED...,Phosphate,0.00000,600 nm,el_mylogic,[O-][P]([O-])([O-])=O,1.54,Refractive index (~600 nm)
3,10.1039/B717069F_© The Royal Society of Chemis...,NaN,NaN,NaN,BK7,0.00000,NaN,snowball,NaN,1.518,refractive index
4,10.1529/biophysj.106.094946,10/23/2008,Biophysical Journal,QUANTITATIVECHARACTERIZATIONBIOLOGICALLIQUIDSF...,ethanol,0.00126,NaN,el_cde_text,CCO,1.35434 ± 0.00126,refractive index


In [12]:
missing_by_row = data.isnull().sum(axis=1)
sorted_rows = data.loc[missing_by_row.sort_values(ascending=False).index]
print("Top 10 rows with the most missing values:")
print(sorted_rows.head(10))

Top 10 rows with the most missing values:
                                                    DOI Date Journal Title  \
3056  10.1016/S0963-9969(01)00105-3Food Research Int...  NaN     NaN   NaN   
2553                                 10.1039/C6AN00509H  NaN     NaN   NaN   
1481                                 10.1039/C7AN01576C  NaN     NaN   NaN   
2999                                 10.1039/C6TC02368A  NaN     NaN   NaN   
4034                                 10.1039/C0CP02270E  NaN     NaN   NaN   
2570       10.1016/j.renene.2018.02.018Renewable Energy  NaN     NaN   NaN   
1443                                 10.1039/C8SC04479A  NaN     NaN   NaN   
4052                                 10.1039/C2JM14369K  NaN     NaN   NaN   
4059  10.1016/S1044-5803(03)00075-5Materials Charact...  NaN     NaN   NaN   
4060                                   10.1039/B414064H  NaN     NaN   NaN   

                         Name  measurement_error measurement_wavelength  \
3056                 gly

Oops, it seems like in some cases in the column 'DOI' the Journal name sticks to the DOI, let's fix it

### Fixing DOI column

Let's have one doi as an example and try to fix it

In [13]:
print(data.loc[3056, 'DOI'])

10.1016/S0963-9969(01)00105-3Food Research International


Let's also convert the 'DOI' column into a list so it would be more comfy to work with ir

In [14]:
DOI_column = data['DOI'].tolist()

To separate the DOI from all the other unnecessary stuff we have in this column, let's use a regular expression

In [15]:
DOI_pattern = re.compile(r'^10\.\d{4,9}\/[-._;()\/:A-Z0-9]+(?=_)')

In [17]:
DOI_example = '10.1016/S0963-9969(01)00105-3Food Research International'
DOI_example

'10.1016/S0963-9969(01)00105-3Food Research International'

In [18]:
DOI_match_example = re.match(DOI_pattern, DOI_example)

In [19]:
DOI_match_example

Okay, it worked. Now let's try it on the whole column

For that, let's first create a function which will check if the doi valid or not 

In [10]:
def is_valid_doi(doi_str:str) -> bool:
    """
    Check if a DOI is valid and corresponds to an article on the internet.
    
    Args:
        doi_str (str): A string representing the DOI to be checked.
        
    Returns:
        bool: True if the DOI is valid and corresponds to an article with metadata available on the internet, False otherwise.
        
    Example Usage:
        >>> is_valid_doi('10.1016/j.jacc.2020.02.068')
        True
    """
    # Construct the API URL for the DOI
    url = f"https://api.crossref.org/works/{doi_str}"
    
    # Make an HTTP request to the API
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Extract the metadata from the response
        metadata = response.json()["message"]
        # Check if the metadata contains a title (i.e., the DOI is valid)
        if "title" in metadata:
            return True
    # If the request failed or the metadata does not contain a title, the DOI is invalid
    return False

Next let's create a function which will clean the DOI column

In [8]:
def clean_DOI(initial_DOI_list: List[str]) -> List[str]:
    """
    Clean a list of DOIs by removing invalid DOIs and keeping only valid DOIs with metadata available on the internet.

    Args:
        initial_DOI_list (List[str]): A list of strings representing the DOIs to be cleaned.

    Returns:
        List[str]: A list of strings representing the cleaned DOIs. Each element of the returned list is either a valid DOI or the string 'invalid' if the DOI is not valid or does not have metadata available on the internet.

    Example Usage:
        >>> clean_DOI(['10.1016/j.jacc.2020.02.068', '10.3390/bs10010012', '10.1038/nature12373', '10.1162/REST_a_00136'])
        ['10.1016/j.jacc.2020.02.068', '10.3390/bs10010012', 'invalid', 'invalid']
    """
    DOI_pattern = re.compile(r'^10\.\d{4,9}\/[-._;()\/:A-Z0-9]+', flags=re.IGNORECASE)
    cleaned_DOI_list = []
    for DOI in tqdm(initial_DOI_list):
        DOI_match = re.findall(DOI_pattern, DOI)
        if DOI_match:
            DOI_cleaned = DOI_match[0].strip('_')
            if is_valid_doi(DOI_cleaned):
                cleaned_DOI_list.append(DOI_cleaned)
            else:
                cleaned_DOI_list.append('invalid')
        else:
            cleaned_DOI_list.append('invalid')
            print(DOI)
        # Sleep for 0.125 seconds
        time.sleep(0.125)
    return cleaned_DOI_list

In [22]:
DOI_new = clean_DOI(DOI_column)

100%|██████████████████████████████████████████████████████████████████████████████| 5000/5000 [48:36<00:00,  1.71it/s]


In [23]:
print(len(DOI_column))
print(len(DOI_new))

5000
5000


In [24]:
DOI_new.count('invalid')

173

After all, there are still 173 invalid DOI's. Let's try to use another regular expressin on those invalid DOI's

In [26]:
invalid_ids = [i for i, x in enumerate(DOI_new) if x == 'invalid']
print(invalid_ids)

[49, 71, 75, 86, 97, 108, 114, 131, 136, 221, 231, 389, 416, 485, 516, 632, 634, 637, 653, 657, 721, 731, 747, 812, 850, 904, 913, 921, 970, 973, 999, 1087, 1099, 1114, 1118, 1143, 1158, 1178, 1195, 1221, 1267, 1349, 1372, 1392, 1413, 1435, 1475, 1487, 1512, 1518, 1519, 1533, 1543, 1548, 1559, 1569, 1576, 1581, 1662, 1663, 1748, 1755, 1775, 1805, 1858, 1882, 1911, 1936, 1953, 1961, 1987, 2022, 2042, 2055, 2057, 2139, 2180, 2237, 2265, 2414, 2498, 2526, 2536, 2557, 2570, 2608, 2687, 2717, 2722, 2787, 2790, 2818, 2830, 2872, 2873, 2903, 3031, 3056, 3084, 3095, 3108, 3155, 3157, 3162, 3178, 3185, 3232, 3306, 3315, 3344, 3377, 3411, 3412, 3413, 3445, 3462, 3518, 3537, 3555, 3565, 3577, 3588, 3598, 3608, 3614, 3682, 3704, 3734, 3746, 3814, 3858, 3914, 3926, 3946, 3947, 3963, 4031, 4056, 4059, 4062, 4092, 4127, 4152, 4160, 4161, 4170, 4245, 4249, 4264, 4272, 4278, 4282, 4330, 4336, 4371, 4391, 4420, 4477, 4500, 4520, 4545, 4551, 4578, 4628, 4681, 4737, 4768, 4885, 4932, 4933, 4945, 4961, 497

In [27]:
invalid_elements = [DOI_column[i] for i in invalid_ids]
print(invalid_elements, invalid_ids)

['10.1016/j.jallcom.2017.03.270Journal of Alloys and Compounds', '10.1016/j.optcom.2015.04.046Optics Communications', '10.1016/j.tsf.2016.01.038Thin Solid Films', '10.1016/S0167-9317(02)01012-2Microelectronic Engineering', '10.1016/j.chroma.2015.07.062Journal of Chromatography A', '10.1016/j.talanta.2015.11.051Talanta', '10.1038/ncomms8', '10.1016/S0021-9673(00)00517-3Journal of Chromatography A', '10.1016/j.mee.2004.03.068Microelectronic Engineering', '10.1016/S0022-3093(02)00967-5Journal of Non-Crystalline Solids', '10.1016/j.ijleo.2018.04.126Optik', '10.1016/j.carbon.2018.01.009Carbon', '10.1016/j.optmat.2004.02.014Optical Materials', '10.1016/j.expthermflusci.2018.02.036Experimental Thermal and Fluid Science', '10.1016/j.bios.2016.05.082Biosensors and Bioelectronics', '10.1016/j.yofte.2018.07.003Optical Fiber Technology', '10.1016/S0168-9002(01)00872-5Nuclear Instruments and Methods in Physics Research Section A: Accelerators, Spectrometers, Detectors and Associated Equipment', '10

First of all let's save our dataframe in case something will go wrong

In [28]:
data.iloc[invalid_ids,:].to_csv("data_2_invalid.tsv", sep='\t')

In [29]:
pwd

'C:\\Users\\bocha\\Algorithms and Big Data in Chemistry and Materials'

New pattern will make sure the DOI ends up with digit

In [30]:
DOI_no_invalid_elements = [re.findall(r'^10\.\d{4,9}\/[-._;()\/:A-Z0-9]+\d', invalid_element) for invalid_element in invalid_elements]

In [31]:
DOI_no_invalid_elements

[[],
 [],
 [],
 ['10.1016/S0167-9317(02)01012-2'],
 [],
 [],
 [],
 ['10.1016/S0021-9673(00)00517-3'],
 [],
 ['10.1016/S0022-3093(02)00967-5'],
 [],
 [],
 [],
 [],
 [],
 [],
 ['10.1016/S0168-9002(01)00872-5'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['10.1016/S0927-0248(01)00058-7'],
 ['10.1016/S0009-2614(00)00534-0'],
 ['10.1016/S0040-6090(00)01205-0'],
 [],
 ['10.1163/15685520360685983'],
 ['10.1016/S0022-2313(01)00371-4'],
 [],
 [],
 [],
 ['10.1016/S0021-9797(03)00452-1'],
 ['10.1016/S0040-6090(00)00712-4'],
 [],
 ['10.1016/S0030-3992(02)00073-7'],
 [],
 ['10.1016/S1369-8001(00)00054-8'],
 [],
 ['10.1016/S1386-9477(02)01046-9'],
 [],
 [],
 ['10.1016/S0146-6410(02)00157-6'],
 ['10.1016/S0168-9002(01)01946-5'],
 ['10.1016/S0168-583X(01)01292-7'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['10.1016/S0032-5910(02)00212-7'],
 [],
 [],
 [],
 [],
 [],
 [],
 ['10.1016/S0925-4005(03)00257-0'],
 [],
 ['10.1016/S0022-3093(02)00921-3'],
 [],
 ['10.1016/S00

In [32]:
status_DOI = [is_valid_doi(DOI_no_invalid_element[0]) for DOI_no_invalid_element in tqdm(DOI_no_invalid_elements) if DOI_no_invalid_element]

100%|████████████████████████████████████████████████████████████████████████████████| 173/173 [00:26<00:00,  6.42it/s]


In [33]:
status_DOI

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 True]

In [34]:
DOI_pattern_2 = re.compile(r'^10\.\d{4,9}\/[-._;()\/:A-Z0-9]+\d', flags=re.IGNORECASE)

In [ ]:
#DOI_new_backup = DOI_new.copy()

In [ ]:
#with open('DOI_new_backup.txt', 'w') as DOI_file:
    #for DOI in DOI_new_backup:
        #DOI_file.write(DOI+'\n')

In [35]:
yet_invalid_DOI = {}
for id_x,DOI in tqdm(enumerate(DOI_new)):
    if DOI == 'invalid':
        DOI_to_cor = DOI_column[id_x]
        DOI_pot_cor = re.findall(DOI_pattern_2, DOI_to_cor)
        if len(DOI_pot_cor)>0:
            if is_valid_doi(DOI_pot_cor[0]):
                DOI_new[id_x] = DOI_pot_cor[0]
            else:
                yet_invalid_DOI[id_x]=DOI_to_cor
        else:
            yet_invalid_DOI[id_x]=DOI_to_cor

5000it [01:16, 65.71it/s] 


We have yet still invalid DOI, let's take a look at them

In [36]:
yet_invalid_DOI

{114: '10.1038/ncomms8',
 136: '10.1016/j.mee.2004.03.068Microelectronic Engineering',
 2057: '10.1016/S0026-2692(03)00137-XMicroelectronics Journal',
 2790: '10.1016/j.snb.2004.06.015',
 3095: '10.1016/S1350-4495(99)00047-XInfrared Physics & Technology',
 3178: '10.3389/fpls.2014.00',
 3555: '10.1002/jbio.201700',
 4420: '10.1016/S0925-4005(99)00427-XSensors and Actuators B: Chemical',
 4500: '10.1063/1.4765',
 4681: '10.1016/S0038-092X(00)00013-XSolar Energy'}

In [37]:
len(DOI_new)

5000

In [38]:
DOI_new.count('invalid')

10

In [ ]:
#DOI_new = DOI_new_backup.copy()

In [39]:
yet_invalid_DOI = {114: '10.1038/ncomms8',
 136: '10.1016/j.mee.2004.03.068Microelectronic Engineering',
 2057: '10.1016/S0026-2692(03)00137-XMicroelectronics Journal',
 2790: '10.1016/j.snb.2004.06.015',
 3095: '10.1016/S1350-4495(99)00047-XInfrared Physics & Technology',
 3178: '10.3389/fpls.2014.00',
 3555: '10.1002/jbio.201700',
 4420: '10.1016/S0925-4005(99)00427-XSensors and Actuators B: Chemical',
 4500: '10.1063/1.4765',
 4681: '10.1016/S0038-092X(00)00013-XSolar Energy'}

There are only 10 of them, so let's check em and append manually

In [40]:
DOI_new[114] = 'NA'
DOI_new[136] = '10.1016/j.mee.2004.03.068'
DOI_new[2057] = '10.1016/S0026-2692(03)00137-X'
DOI_new[2790] = '10.1016/j.snb.2004.06.015'
DOI_new[3095] = '10.1016/S1350-4495(99)00047-X'
DOI_new[3178] = 'NA'
DOI_new[3555] = 'NA'
DOI_new[4420] = '10.1016/S0925-4005(99)00427-X'
DOI_new[4500] = 'NA'
DOI_new[4681] = '10.1016/S0038-092X(00)00013-X'

In [41]:
data['DOI'] = DOI_new

In [42]:
data.to_csv('data_2_DOI_new.tsv', sep='\t')

## Getting missing values

In [43]:
pip install crossref-commons

Note: you may need to restart the kernel to use updated packages.


In [44]:
import crossref_commons.retrieval

In [45]:
tmp = crossref_commons.retrieval.get_publication_as_json('10.1016/j.jallcom.2017.03.270')

In [46]:
for key,value in tmp.items():
    print(key)

indexed
reference-count
publisher
license
funder
content-domain
short-container-title
published-print
DOI
type
created
page
update-policy
source
is-referenced-by-count
title
prefix
volume
author
member
reference
container-title
original-title
language
link
deposited
score
resource
subtitle
short-title
issued
references-count
alternative-id
URL
relation
ISSN
issn-type
subject
published
assertion


In [47]:
tmp

{'indexed': {'date-parts': [[2022, 12, 31]],
  'date-time': '2022-12-31T11:11:56Z',
  'timestamp': 1672485116510},
 'reference-count': 22,
 'publisher': 'Elsevier BV',
 'license': [{'start': {'date-parts': [[2017, 7, 1]],
    'date-time': '2017-07-01T00:00:00Z',
    'timestamp': 1498867200000},
   'content-version': 'tdm',
   'delay-in-days': 0,
   'URL': 'https://www.elsevier.com/tdm/userlicense/1.0/'}],
 'funder': [{'DOI': '10.13039/501100004085',
   'name': 'Ministry of Education, Science and Technology',
   'doi-asserted-by': 'publisher',
   'award': ['NRF-2015R1A1A1A05027848']},
  {'DOI': '10.13039/501100003725',
   'name': 'National Research Foundation of Korea',
   'doi-asserted-by': 'publisher'}],
 'content-domain': {'domain': ['elsevier.com', 'sciencedirect.com'],
  'crossmark-restriction': True},
 'short-container-title': ['Journal of Alloys and Compounds'],
 'published-print': {'date-parts': [[2017, 7]]},
 'DOI': '10.1016/j.jallcom.2017.03.270',
 'type': 'journal-article',
 

In [48]:
tmp['indexed']['date-parts'][0]

[2022, 12, 31]

In [49]:
publish_date = f"{tmp['indexed']['date-parts'][0][1]}/{tmp['indexed']['date-parts'][0][2]}/{tmp['indexed']['date-parts'][0][0]}"

In [50]:
publish_date

'12/31/2022'

In [51]:
article_title = tmp['title'][0]

In [52]:
article_title

'Improving light extraction in light-emitting diodes using zinc-tin-oxide layers'

In [53]:
print(tmp['short-container-title'])
print(tmp['container-title'])
print(tmp['original-title'])

['Journal of Alloys and Compounds']
['Journal of Alloys and Compounds']
[]


In [54]:
journal_title = tmp['short-container-title'][0]

In [ ]:
journal_title

In [55]:
#Let's count how many rows have incorrect DOIs:
data['DOI'].value_counts()['NA']

4

In [56]:
data = data.drop(data[data['DOI'] == 'NA'].index)
len(data)

4996

### Getting rid of duplicates

In [57]:
#let's check if there are any duplicates in the df:
data.duplicated().sum()

21

In [58]:
duplicates = data[data.duplicated(keep=False)]
print(duplicates)

                                 DOI        Date  \
10    10.1016/j.atmosres.2006.02.003    6/2/2006   
47         10.1016/j.saa.2019.01.086   1/29/2019   
103     10.1016/j.jtusci.2016.03.002   4/11/2016   
229     10.1016/j.jtusci.2016.03.002   4/11/2016   
1100  10.1016/j.foodchem.2013.12.085    1/3/2014   
1168              10.1039/C3RA41435C    8/5/2013   
1179  10.1016/j.atmosres.2006.02.003    6/2/2006   
1529   10.1016/S0925-3467(02)00231-8  12/11/2002   
1544    10.1016/j.molliq.2016.04.011   4/30/2016   
1614  10.1016/j.foodchem.2013.12.085    1/3/2014   
1635     10.1016/j.jksus.2012.12.004  12/27/2012   
1690     10.1016/j.fluid.2015.03.040   3/28/2015   
1696    10.1016/j.molliq.2016.04.011   4/30/2016   
1719     10.1016/j.physb.2006.07.020    9/1/2006   
1837       10.1016/j.saa.2019.01.086   1/29/2019   
1939     10.1016/j.jksus.2012.12.004  12/27/2012   
2273    10.1016/j.molliq.2013.09.035  10/17/2013   
2309   10.1016/S0925-3467(02)00231-8  12/11/2002   
2379        

In [59]:
unique_data = data.drop_duplicates()
len(unique_data)

4975

### Filling in title, journal names and date

In [60]:
#to do: сделать функцию
#проверка1: АПИ вернуло что-то (лен тмп больше 0)
#проверка2: indexed, title, short-container-title ЕСТЬ -> (try (выполнить) exept ('NA'))
#сделать словарь: ключ - DOI, значения - лист(indexed, title, short-container-title) -> pandas.df -> примёрджить к нашей data по DOI

In [61]:
print("Missing values distribution by column: ")
print(data.isnull().mean())
print("")

Missing values distribution by column: 
DOI                       0.000000
Date                      0.083467
Journal                   0.083467
Title                     0.083467
Name                      0.001001
measurement_error         0.000000
measurement_wavelength    0.885709
measurement_method        0.000000
normalised_name           0.411729
raw_value                 0.000000
specifier                 0.000000
dtype: float64



In [62]:
data.head()

,DOI,Date,Journal,Title,Name,measurement_error,measurement_wavelength,measurement_method,normalised_name,raw_value,specifier
0,10.1016/j.optlaseng.2008.09.005,1/14/2009,Optics and Lasers in Engineering,AHIGHPRECISIONMEASUREMENTTECHNIQUEFOREVALUATIN...,Alcohol,0.00000,NaN,el_mylogic,CCO,1.34749,Refractive index (n)
1,10.1016/j.jiec.2018.12.038,12/29/2018,Journal of Industrial and Engineering Chemistry,IMPROVINGUNDERSTANDINGSOLVENTEFFECTSINTERMOLEC...,aceticacid,0.00000,NaN,el_mylogic,CC(O)=O,1.7,n
2,10.1016/j.jlumin.2015.02.010,2/14/2015,Journal of Luminescence,SPECTROSCOPICPROPERTIESTELLURITEGLASSESCODOPED...,Phosphate,0.00000,600 nm,el_mylogic,[O-][P]([O-])([O-])=O,1.54,Refractive index (~600 nm)
3,10.1039/B717069F,NaN,NaN,NaN,BK7,0.00000,NaN,snowball,NaN,1.518,refractive index
4,10.1529/biophysj.106.094946,10/23/2008,Biophysical Journal,QUANTITATIVECHARACTERIZATIONBIOLOGICALLIQUIDSF...,ethanol,0.00126,NaN,el_cde_text,CCO,1.35434 ± 0.00126,refractive index


Here we can see, that the most missing data we have is in the columns 'measurement_wavelength', 'normalised_name', but we'll fill them a bit later here. Firstly let's look at the other column - 'Title'. For some reason the name of the articles were imported not as the sentances but as the continuous sequence of large letters. Also there is missing values in the 'Date' and 'Journal' columns which we should also fill in. Now when we know that all the rows are unique and valid, let's collect this data from the articles with the function.

In [63]:
unique_data.to_csv('data_copy.csv', sep=',', index=False)

In [64]:
missing_name_count = len(unique_data[unique_data['Name'].isnull()])
print(missing_name_count)
missing_name_rows = data[data['Name'].isnull()]
print(missing_name_rows)

5
                              DOI        Date  \
13     10.1016/j.jcis.2009.08.028   8/21/2009   
154   10.1016/j.jdent.2013.09.007   10/3/2013   
2805    10.1016/j.tca.2016.05.013   5/26/2016   
3075    10.1016/j.saa.2007.04.016   4/24/2007   
3151             10.1039/B508730A  11/22/2005   

                                                Journal  \
13             Journal of Colloid and Interface Science   
154                                Journal of Dentistry   
2805                                 Thermochimica Acta   
3075  Spectrochimica Acta Part A: Molecular and Biom...   
3151                           New Journal of Chemistry   

                                                  Title Name  \
13    SPECTRALPARAMETERSHAMAKERCONSTANTSSILICONHYDRI...  NaN   
154   EFFECTTHREEYEARCONSUMPTIONERYTHRITOLXYLITOLSOR...  NaN   
2805  THERMODYNAMICPROPERTIES1BUTYL3METHYLIMIDAZOLIU...  NaN   
3075  SOLVENTEFFECTABSORPTIONFLUORESCENCESPECTRACOUM...  NaN   
3151  Silver organosol: synt

In [65]:
missing_name_rows = data[data['Name'].isnull()]
missing_name_rows.loc[missing_name_rows['Name'].isnull(), 'Name'] = ['cyclopentane', '1,2,3,4,5-pentakis(hydroxymethyl)cyclohexane', 'glycine', 'decyl alcohol', 'N,N-dimethylformamide']
print(missing_name_rows)

                              DOI        Date  \
13     10.1016/j.jcis.2009.08.028   8/21/2009   
154   10.1016/j.jdent.2013.09.007   10/3/2013   
2805    10.1016/j.tca.2016.05.013   5/26/2016   
3075    10.1016/j.saa.2007.04.016   4/24/2007   
3151             10.1039/B508730A  11/22/2005   

                                                Journal  \
13             Journal of Colloid and Interface Science   
154                                Journal of Dentistry   
2805                                 Thermochimica Acta   
3075  Spectrochimica Acta Part A: Molecular and Biom...   
3151                           New Journal of Chemistry   

                                                  Title  \
13    SPECTRALPARAMETERSHAMAKERCONSTANTSSILICONHYDRI...   
154   EFFECTTHREEYEARCONSUMPTIONERYTHRITOLXYLITOLSOR...   
2805  THERMODYNAMICPROPERTIES1BUTYL3METHYLIMIDAZOLIU...   
3075  SOLVENTEFFECTABSORPTIONFLUORESCENCESPECTRACOUM...   
3151  Silver organosol: synthesis, characterisation ...

In [66]:
unique_data.loc[missing_name_rows.index, 'Name'] = missing_name_rows['Name']
print(unique_data)

                                  DOI        Date  \
0     10.1016/j.optlaseng.2008.09.005   1/14/2009   
1          10.1016/j.jiec.2018.12.038  12/29/2018   
2        10.1016/j.jlumin.2015.02.010   2/14/2015   
3                    10.1039/B717069F         NaN   
4         10.1529/biophysj.106.094946  10/23/2008   
...                               ...         ...   
4995        10.1016/j.sse.2007.06.004   7/30/2007   
4996                 10.1039/B822982A   7/21/2009   
4997          10.1078/0030-4026-00175   11/5/2004   
4998     10.1016/j.optmat.2018.06.013   6/18/2018   
4999    10.1016/S0042-207X(01)00296-2  12/17/2001   

                                              Journal  \
0                    Optics and Lasers in Engineering   
1     Journal of Industrial and Engineering Chemistry   
2                             Journal of Luminescence   
3                                                 NaN   
4                                 Biophysical Journal   
...                  

In [67]:
print("Missing values distribution by column: ")
print(unique_data.isnull().mean())

Missing values distribution by column: 
DOI                       0.000000
Date                      0.083819
Journal                   0.083819
Title                     0.083819
Name                      0.000000
measurement_error         0.000000
measurement_wavelength    0.885628
measurement_method        0.000000
normalised_name           0.411457
raw_value                 0.000000
specifier                 0.000000
dtype: float64



In [68]:
def fill_date_journal_title(data):
    result = {}
    for row in tqdm(data):
        doi, date, journal, title = row
        if not is_valid_doi(doi):
            result[doi] = [date, journal, title]
            continue
        tmp = crossref_commons.retrieval.get_publication_as_json(doi)
        if len(tmp) == 0:
            result[doi] = [date, journal, title]
            continue
        try:
            date_parts = tmp['indexed']['date-parts'][0]
            if date_parts:
                date = f"{date_parts[1]}/{date_parts[2]}/{date_parts[0]}"
        except (KeyError, TypeError, IndexError):
            pass
        try:
            title = tmp['title'][0]
        except (KeyError, TypeError, IndexError):
            pass
        try:
            journal = tmp['short-container-title'][0]
        except (KeyError, TypeError, IndexError):
            pass
        result[doi] = [date, journal, title]
        # Sleep for 0.125 seconds
        time.sleep(0.125)
    return result

In [69]:
unique_data.to_csv('unique_data.csv', index=False, sep=',')

In [70]:
data_list = unique_data[['DOI', 'Date', 'Journal', 'Title']].values.tolist()
data_filled = fill_date_journal_title(data_list)

100%|████████████████████████████████████████████████████████████████████████████| 4975/4975 [1:23:14<00:00,  1.00s/it]


In [72]:
print("Missing values distribution by column: ")
print(unique_data.isnull().mean())

Missing values distribution by column: 
DOI                       0.000000
Date                      0.083819
Journal                   0.083819
Title                     0.083819
Name                      0.000000
measurement_error         0.000000
measurement_wavelength    0.885628
measurement_method        0.000000
normalised_name           0.411457
raw_value                 0.000000
specifier                 0.000000
dtype: float64


In [73]:
data_filled

{'10.1016/j.optlaseng.2008.09.005': ['2/14/2023',
  'Optics and Lasers in Engineering',
  'A high-precision measurement technique for evaluating alcohol concentrations using an optical metrology system based on a position sensing detector'],
 '10.1016/j.jiec.2018.12.038': ['2/13/2023',
  'Journal of Industrial and Engineering Chemistry',
  'Improving understanding of solvent effects on intermolecular interactions in reactive liquid–liquid extraction with Isothermal Titration Calorimetry and molecular modeling'],
 '10.1016/j.jlumin.2015.02.010': ['12/27/2022',
  'Journal of Luminescence',
  'Spectroscopic properties of tellurite glasses co-doped with Er3+ and Yb3+'],
 '10.1039/B717069F': ['3/7/2023',
  'J. Mater. Chem.',
  'High refractive index polyimide–nanocrystalline-titania hybrid optical materials'],
 '10.1529/biophysj.106.094946': ['3/7/2023',
  'Biophysical Journal',
  'Quantitative Characterization of Biological Liquids for Third-Harmonic Generation Microscopy'],
 '10.1039/B211

In [74]:
import json
# open file in write mode
with open("data_filled.json", "w") as outfile:
    # write dictionary to file in JSON format
    json.dump(data_filled, outfile)

In [75]:
len(data_filled)

2812

In [87]:
# create a copy of the original DataFrame
new_df = unique_data.copy()

# iterate over the dictionary and update the DataFrame
for doi, values in data_filled.items():
    if doi in new_df.index:
        new_df.loc[doi, ['Date', 'Journal', 'Title']] = values


In [89]:
for i, row in unique_data.iterrows():
    doi = row['DOI']
    if doi in data_filled:
        data, journal, title = data_filled[doi]
        unique_data.at[i, 'Data'] = data
        unique_data.at[i, 'Journal'] = journal
        unique_data.at[i, 'Title'] = title

In [90]:
unique_data

,DOI,Date,Journal,Title,Name,measurement_error,measurement_wavelength,measurement_method,normalised_name,raw_value,specifier,Data
0,10.1016/j.optlaseng.2008.09.005,1/14/2009,Optics and Lasers in Engineering,A high-precision measurement technique for eva...,Alcohol,0.00000,NaN,el_mylogic,CCO,1.34749,Refractive index (n),2/14/2023
1,10.1016/j.jiec.2018.12.038,12/29/2018,Journal of Industrial and Engineering Chemistry,Improving understanding of solvent effects on ...,aceticacid,0.00000,NaN,el_mylogic,CC(O)=O,1.7,n,2/13/2023
2,10.1016/j.jlumin.2015.02.010,2/14/2015,Journal of Luminescence,Spectroscopic properties of tellurite glasses ...,Phosphate,0.00000,600 nm,el_mylogic,[O-][P]([O-])([O-])=O,1.54,Refractive index (~600 nm),12/27/2022
3,10.1039/B717069F,NaN,J. Mater. Chem.,High refractive index polyimide–nanocrystallin...,BK7,0.00000,NaN,snowball,NaN,1.518,refractive index,3/7/2023
4,10.1529/biophysj.106.094946,10/23/2008,Biophysical Journal,Quantitative Characterization of Biological Li...,ethanol,0.00126,NaN,el_cde_text,CCO,1.35434 ± 0.00126,refractive index,3/7/2023
...,...,...,...,...,...,...,...,...,...,...,...,...
4995,10.1016/j.sse.2007.06.004,7/30/2007,Solid-State Electronics,Radiative line width of a single-impurity mole...,Biphenyl,0.00000,NaN,el_cde_tables,c1ccc(cc1)c2ccccc2,1.68,Refractive indices,3/29/2022
4996,10.1039/B822982A,7/21/2009,Lab Chip,Tunable Liquid Gradient Refractive Index (L-GR...,CaCl2,0.00000,NaN,rsc_cde_text,"[['Ca', 1.0], ['Cl', 2.0]]",∼1.41,nD,2/19/2023
4997,10.1078/0030-4026-00175,11/5/2004,Optik,Optimization of conducting polymer thin film o...,He-Ne,0.00000,NaN,el_cde_text,"[['He-', 1.0], ['Ne', 1.0]]",1.717,refractive index,1/27/2023
4998,10.1016/j.optmat.2018.06.013,6/18/2018,Optical Materials,Enhanced photoelectric performance of nanorod ...,ITO,0.00000,NaN,el_cde_text,"[['IT', 1.0], ['O', 1.0]]",1.92,refractive index,11/18/2022


In [91]:
print("Missing values distribution by column: ")
print(unique_data.isnull().mean())

Missing values distribution by column: 
DOI                       0.000000
Date                      0.083819
Journal                   0.000804
Title                     0.000201
Name                      0.000000
measurement_error         0.000000
measurement_wavelength    0.885628
measurement_method        0.000000
normalised_name           0.411457
raw_value                 0.000000
specifier                 0.000000
Data                      0.000201
dtype: float64


In [94]:
missing_data_rows = unique_data.loc[unique_data['Data'].isnull()]
print(missing_data_rows)


                           DOI Date Journal Title     Name  measurement_error  \
136  10.1016/j.mee.2004.03.068  NaN     NaN   NaN  ethanol                0.0   

    measurement_wavelength measurement_method normalised_name raw_value  \
136                    NaN           snowball             NaN      1.36   

    specifier Data  
136         n  NaN  


In [97]:
unique_data.at[136, 'Data'] = '06/22/2004'

In [95]:
missing_journal_rows = unique_data.loc[unique_data['Journal'].isnull()]
print(missing_journal_rows)


                                DOI Date Journal  \
136       10.1016/j.mee.2004.03.068  NaN     NaN   
2237  10.1016/S0016-2361(99)00288-4  NaN     NaN   
3413  10.1016/S0956-5663(00)00072-5  NaN     NaN   
4278  10.1016/S1388-2481(02)00256-4  NaN     NaN   

                                                  Title             Name  \
136                                                 NaN          ethanol   
2237  Optimization of the HPLC separation of aromati...  isopropyl ether   
3413  High sensitivity waveguide grating sensor base...             TiO2   
4278  Electrosynthesis of polyphenylpyrrole coated s...           silver   

      measurement_error measurement_wavelength measurement_method  \
136                 0.0                    NaN           snowball   
2237                0.0                    NaN           snowball   
3413                0.0                    NaN           snowball   
4278                0.0                    NaN           snowball   

            

In [98]:
unique_data.at[136, 'Journal'] = 'Microelectronic Engineering'
unique_data.at[2237, 'Journal'] = 'Fuel'
unique_data.at[3413, 'Journal'] = 'Biosensors and Bioelectronics'
unique_data.at[4278, 'Journal'] = 'Electrochemistry Communications'

In [96]:
missing_title_rows = unique_data.loc[unique_data['Title'].isnull()]
print(missing_title_rows)

                           DOI Date Journal Title     Name  measurement_error  \
136  10.1016/j.mee.2004.03.068  NaN     NaN   NaN  ethanol                0.0   

    measurement_wavelength measurement_method normalised_name raw_value  \
136                    NaN           snowball             NaN      1.36   

    specifier Data  
136         n  NaN  


In [99]:
unique_data.at[136, 'Title'] = 'On-chip optical components and microfluidic systems'

In [104]:
unique_data

,DOI,Date,Journal,Title,Name,measurement_error,measurement_wavelength,measurement_method,normalised_name,raw_value,specifier,Data
0,10.1016/j.optlaseng.2008.09.005,1/14/2009,Optics and Lasers in Engineering,A high-precision measurement technique for eva...,Alcohol,0.00000,NaN,el_mylogic,CCO,1.34749,Refractive index (n),2/14/2023
1,10.1016/j.jiec.2018.12.038,12/29/2018,Journal of Industrial and Engineering Chemistry,Improving understanding of solvent effects on ...,aceticacid,0.00000,NaN,el_mylogic,CC(O)=O,1.7,n,2/13/2023
2,10.1016/j.jlumin.2015.02.010,2/14/2015,Journal of Luminescence,Spectroscopic properties of tellurite glasses ...,Phosphate,0.00000,600 nm,el_mylogic,[O-][P]([O-])([O-])=O,1.54,Refractive index (~600 nm),12/27/2022
3,10.1039/B717069F,NaN,J. Mater. Chem.,High refractive index polyimide–nanocrystallin...,BK7,0.00000,NaN,snowball,NaN,1.518,refractive index,3/7/2023
4,10.1529/biophysj.106.094946,10/23/2008,Biophysical Journal,Quantitative Characterization of Biological Li...,ethanol,0.00126,NaN,el_cde_text,CCO,1.35434 ± 0.00126,refractive index,3/7/2023
...,...,...,...,...,...,...,...,...,...,...,...,...
4995,10.1016/j.sse.2007.06.004,7/30/2007,Solid-State Electronics,Radiative line width of a single-impurity mole...,Biphenyl,0.00000,NaN,el_cde_tables,c1ccc(cc1)c2ccccc2,1.68,Refractive indices,3/29/2022
4996,10.1039/B822982A,7/21/2009,Lab Chip,Tunable Liquid Gradient Refractive Index (L-GR...,CaCl2,0.00000,NaN,rsc_cde_text,"[['Ca', 1.0], ['Cl', 2.0]]",∼1.41,nD,2/19/2023
4997,10.1078/0030-4026-00175,11/5/2004,Optik,Optimization of conducting polymer thin film o...,He-Ne,0.00000,NaN,el_cde_text,"[['He-', 1.0], ['Ne', 1.0]]",1.717,refractive index,1/27/2023
4998,10.1016/j.optmat.2018.06.013,6/18/2018,Optical Materials,Enhanced photoelectric performance of nanorod ...,ITO,0.00000,NaN,el_cde_text,"[['IT', 1.0], ['O', 1.0]]",1.92,refractive index,11/18/2022


In [110]:
print("Missing values distribution by column: ")
print(unique_data.isnull().mean())

Missing values distribution by column: 
DOI                       0.000000
Date                      0.083819
Journal                   0.000000
Title                     0.000000
Name                      0.000000
measurement_error         0.000000
measurement_wavelength    0.885628
measurement_method        0.000000
normalised_name           0.411457
raw_value                 0.000000
specifier                 0.000000
Data                      0.000000
dtype: float64


In [112]:
missing_data_rows = unique_data.loc[unique_data['Date'].isnull()]
print(missing_data_rows)

                                DOI Date  \
3                  10.1039/B717069F  NaN   
5                  10.1039/B211264G  NaN   
11               10.1039/C3CP55098B  NaN   
23               10.1039/C3TC30313F  NaN   
36               10.1039/C1CP21073D  NaN   
...                             ...  ...   
4932  10.1016/S0003-2670(00)00927-2  NaN   
4933   10.1016/j.optcom.2016.11.031  NaN   
4945  10.1016/j.polymer.2018.03.018  NaN   
4961      10.1016/j.snb.2018.03.136  NaN   
4977  10.1016/S0967-0637(99)00085-0  NaN   

                                                Journal  \
3                                       J. Mater. Chem.   
5                               Phys. Chem. Chem. Phys.   
11                              Phys. Chem. Chem. Phys.   
23                                    J. Mater. Chem. C   
36                              Phys. Chem. Chem. Phys.   
...                                                 ...   
4932                             Analytica Chimica Acta   

In [113]:
unique_data.to_csv('unique_data_2.0.tsv', sep='\t')

### Filling in date

In [5]:
def paste_date_from_doi(unique_data, data_filled):
    """
    Searches for an article from the 'unique_data' DataFrame using its DOI from the 'DOI' column, 
    and then pastes its date in the 'Date' column if it's missing. The function also checks if the 
    DOI is valid using the 'is_valid_doi()' function. If the DOI is not valid or there is no date 
    available, the function pastes 'NaN' in the 'Date' column.
    
    Args:
        unique_data (pandas.DataFrame): The DataFrame to search for the article and update the 'Date' column.
        data_filled (dict): A dictionary containing the article metadata with DOIs as keys and a list 
                            containing the metadata values as values.
        
    Returns:
        pandas.DataFrame: The updated DataFrame with the date pasted in the 'Date' column.
    """
    for doi, metadata in tqdm(data_filled.items()):
        # Check if the DOI is valid
        if is_valid_doi(doi):
            # Find the rows where the DOI matches and the 'Date' column is empty
            match_rows = (unique_data['DOI'] == doi) & (unique_data['Date'].isnull())
            # If there is a match, paste the date in the 'Date' column
            if match_rows.any():
                unique_data.loc[match_rows, 'Date'] = metadata[0]
    # Replace missing values with NaN
    unique_data['Date'] = unique_data['Date'].fillna(value='NaN')
    return unique_data

In [6]:
import json
with open('data_filled.json', 'r') as f:
    data_to_fill = json.load(f)

In [7]:
data = pd.read_csv('data_filled_2.tsv', sep='\t')

In [8]:
print("Missing values distribution by column: ")
print(data.isnull().mean())

Missing values distribution by column: 
Unnamed: 0                0.000000
DOI                       0.000000
Date                      0.083819
Journal                   0.000000
Title                     0.000000
Name                      0.000000
measurement_error         0.000000
measurement_wavelength    0.885628
measurement_method        0.000000
normalised_name           0.411457
raw_value                 0.000000
specifier                 0.000000
Data                      0.000000
dtype: float64


In [11]:
data_filled = paste_date_from_doi(data, data_to_fill)

100%|██████████████████████████████████████████████████████████████████████████████| 2812/2812 [23:52<00:00,  1.96it/s]


In [12]:
print("Missing values distribution by column: ")
print(data_filled.isnull().mean())

Missing values distribution by column: 
Unnamed: 0                0.000000
DOI                       0.000000
Date                      0.000000
Journal                   0.000000
Title                     0.000000
Name                      0.000000
measurement_error         0.000000
measurement_wavelength    0.885628
measurement_method        0.000000
normalised_name           0.411457
raw_value                 0.000000
specifier                 0.000000
Data                      0.000000
dtype: float64


In [13]:
data_filled.to_csv('data_filled_EXTRA.tsv', sep='\t')

### Filling in SMILES

In [17]:
# Set up the PubChem API URL and parameters
url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/"
params = {"property": "CanonicalSMILES"}

# Create an empty list to store the SMILES
smiles = []

# Loop through each compound name and get the SMILES from the PubChem API
for name in tqdm(data_filled["Name"]):
    try:
        # Make a request to the PubChem API and extract the SMILES
        response = requests.get(url + name + "/property/" + params["property"] + "/JSON")
        response_json = response.json()
        smile = response_json["PropertyTable"]["Properties"][0]["CanonicalSMILES"]
        smiles.append(smile)
    except:
        # If there is an error, append a NaN value to the list
        smiles.append(None)

# Add the SMILES to the data frame
data_filled["SMILES"] = smiles

# Save the updated data frame to a new CSV file
data_filled.to_csv("data_with_smiles.tsv", sep='\t')

100%|██████████████████████████████████████████████████████████████████████████████| 4975/4975 [43:02<00:00,  1.93it/s]


In [47]:
data_filled = pd.read_csv('data_with_smiles.tsv', sep='\t')

In [48]:
data_filled.head()

,Unnamed: 0.1,Unnamed: 0,DOI,Date,Journal,Title,Name,measurement_error,measurement_wavelength,measurement_method,normalised_name,raw_value,specifier,Data,SMILES
0,0,0,10.1016/j.optlaseng.2008.09.005,1/14/2009,Optics and Lasers in Engineering,A high-precision measurement technique for eva...,Alcohol,0.00000,NaN,el_mylogic,CCO,1.34749,Refractive index (n),2/14/2023,CCO
1,1,1,10.1016/j.jiec.2018.12.038,12/29/2018,Journal of Industrial and Engineering Chemistry,Improving understanding of solvent effects on ...,aceticacid,0.00000,NaN,el_mylogic,CC(O)=O,1.7,n,2/13/2023,NaN
2,2,2,10.1016/j.jlumin.2015.02.010,2/14/2015,Journal of Luminescence,Spectroscopic properties of tellurite glasses ...,Phosphate,0.00000,600 nm,el_mylogic,[O-][P]([O-])([O-])=O,1.54,Refractive index (~600 nm),12/27/2022,[O-]P(=O)([O-])[O-]
3,3,3,10.1039/B717069F,NaN,J. Mater. Chem.,High refractive index polyimide–nanocrystallin...,BK7,0.00000,NaN,snowball,NaN,1.518,refractive index,3/7/2023,CCOC1=CC2=C(C=C1)C=C(C=C2)C3=NN(C4=NC=NC(=C34)...
4,4,4,10.1529/biophysj.106.094946,10/23/2008,Biophysical Journal,Quantitative Characterization of Biological Li...,ethanol,0.00126,NaN,el_cde_text,CCO,1.35434 ± 0.00126,refractive index,3/7/2023,CCO


In [49]:
print("Missing values distribution by column: ")
print(data_filled.isnull().mean())

Missing values distribution by column: 
Unnamed: 0.1              0.000000
Unnamed: 0                0.000000
DOI                       0.000000
Date                      0.083819
Journal                   0.000000
Title                     0.000000
Name                      0.000000
measurement_error         0.000000
measurement_wavelength    0.885628
measurement_method        0.000000
normalised_name           0.411457
raw_value                 0.000000
specifier                 0.000000
Data                      0.000000
SMILES                    0.311558
dtype: float64


In [50]:
data_filled["normalised_name"].fillna(data_filled["SMILES"], inplace=True)

In [51]:
print("Missing values distribution by column: ")
print(data_filled.isnull().mean())

Missing values distribution by column: 
Unnamed: 0.1              0.000000
Unnamed: 0                0.000000
DOI                       0.000000
Date                      0.083819
Journal                   0.000000
Title                     0.000000
Name                      0.000000
measurement_error         0.000000
measurement_wavelength    0.885628
measurement_method        0.000000
normalised_name           0.129447
raw_value                 0.000000
specifier                 0.000000
Data                      0.000000
SMILES                    0.311558
dtype: float64


In [52]:
data_filled

,Unnamed: 0.1,Unnamed: 0,DOI,Date,Journal,Title,Name,measurement_error,measurement_wavelength,measurement_method,normalised_name,raw_value,specifier,Data,SMILES
0,0,0,10.1016/j.optlaseng.2008.09.005,1/14/2009,Optics and Lasers in Engineering,A high-precision measurement technique for eva...,Alcohol,0.00000,NaN,el_mylogic,CCO,1.34749,Refractive index (n),2/14/2023,CCO
1,1,1,10.1016/j.jiec.2018.12.038,12/29/2018,Journal of Industrial and Engineering Chemistry,Improving understanding of solvent effects on ...,aceticacid,0.00000,NaN,el_mylogic,CC(O)=O,1.7,n,2/13/2023,NaN
2,2,2,10.1016/j.jlumin.2015.02.010,2/14/2015,Journal of Luminescence,Spectroscopic properties of tellurite glasses ...,Phosphate,0.00000,600 nm,el_mylogic,[O-][P]([O-])([O-])=O,1.54,Refractive index (~600 nm),12/27/2022,[O-]P(=O)([O-])[O-]
3,3,3,10.1039/B717069F,NaN,J. Mater. Chem.,High refractive index polyimide–nanocrystallin...,BK7,0.00000,NaN,snowball,CCOC1=CC2=C(C=C1)C=C(C=C2)C3=NN(C4=NC=NC(=C34)...,1.518,refractive index,3/7/2023,CCOC1=CC2=C(C=C1)C=C(C=C2)C3=NN(C4=NC=NC(=C34)...
4,4,4,10.1529/biophysj.106.094946,10/23/2008,Biophysical Journal,Quantitative Characterization of Biological Li...,ethanol,0.00126,NaN,el_cde_text,CCO,1.35434 ± 0.00126,refractive index,3/7/2023,CCO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4970,4970,4995,10.1016/j.sse.2007.06.004,7/30/2007,Solid-State Electronics,Radiative line width of a single-impurity mole...,Biphenyl,0.00000,NaN,el_cde_tables,c1ccc(cc1)c2ccccc2,1.68,Refractive indices,3/29/2022,C1=CC=C(C=C1)C2=CC=CC=C2
4971,4971,4996,10.1039/B822982A,7/21/2009,Lab Chip,Tunable Liquid Gradient Refractive Index (L-GR...,CaCl2,0.00000,NaN,rsc_cde_text,"[['Ca', 1.0], ['Cl', 2.0]]",∼1.41,nD,2/19/2023,[Cl-].[Cl-].[Ca+2]
4972,4972,4997,10.1078/0030-4026-00175,11/5/2004,Optik,Optimization of conducting polymer thin film o...,He-Ne,0.00000,NaN,el_cde_text,"[['He-', 1.0], ['Ne', 1.0]]",1.717,refractive index,1/27/2023,NaN
4973,4973,4998,10.1016/j.optmat.2018.06.013,6/18/2018,Optical Materials,Enhanced photoelectric performance of nanorod ...,ITO,0.00000,NaN,el_cde_text,"[['IT', 1.0], ['O', 1.0]]",1.92,refractive index,11/18/2022,NaN


In [53]:
missing_values = data_filled[data_filled["normalised_name"].isnull()]
missing_values

,Unnamed: 0.1,Unnamed: 0,DOI,Date,Journal,Title,Name,measurement_error,measurement_wavelength,measurement_method,normalised_name,raw_value,specifier,Data,SMILES
7,7,7,10.1016/j.tsf.2018.01.058,2/2/2018,Thin Solid Films,Density and size effects on the thermal conduc...,Al2O3,0.000,20 nm,el_mylogic,NaN,1.71,Index of refraction,2/23/2023,NaN
15,15,15,10.1016/S0378-7788(00)00124-9,5/9/2001,Energy and Buildings,"Experimental results of transparent, reflectiv...",Polycarbonate,0.000,NaN,el_mylogic,NaN,1.55,Refraction index,4/5/2022,NaN
17,17,17,10.1016/j.optmat.2008.10.005,4/26/2009,Optical Materials,CO2 laser annealing on erbium-activated glass–...,SiO2–ZrO2,0.000,0.3 nm,el_mylogic,NaN,1.563,Refractive index at 1.3 μm ± 0.005,11/29/2022,NaN
23,23,23,10.1039/C3TC30313F,NaN,J. Mater. Chem. C,Mapping the structural and optical properties ...,sapphire,0.000,NaN,snowball,NaN,1.33,refraction indices,3/31/2022,NaN
26,26,26,10.1016/j.physleta.2014.02.028,2/28/2014,Physics Letters A,Comb-like optical transmission spectra generat...,TiSi2,0.000,41.35 μm,el_mylogic,NaN,1.809,Refractive index,9/29/2022,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4957,4957,4982,10.1016/j.jct.2011.09.011,9/28/2011,The Journal of Chemical Thermodynamics,Measurement and correlation of the excess mola...,C10H21OH,0.000,NaN,el_mylogic,NaN,1.4374,nD,6/23/2022,NaN
4958,4958,4983,10.1016/j.jnoncrysol.2018.02.007,2/21/2018,Journal of Non-Crystalline Solids,Calculation of the structure and physical prop...,Na2O·2SiO2,0.000,NaN,el_mylogic,NaN,1.5072 [],Refractive (nD),3/6/2023,NaN
4961,4961,4986,10.1016/S0272-8842(01)00005-0,6/12/2001,Ceramics International,The optical waveguide characteristics of highl...,Si ( 111 ),0.000,NaN,el_cde_tables,NaN,1.78,Refractive index,6/27/2022,NaN
4962,4962,4987,10.1016/j.ceramint.2018.08.080,8/9/2018,Ceramics International,Structural and spectroscopic properties of Yb3...,BPYb20,0.001,NaN,el_cde_tables,NaN,1.545 ± 0.001,Refractive index,3/7/2023,NaN


In [56]:
data_filled = data_filled.drop(["Unnamed: 0", "Data", "SMILES", "Unnamed: 0.1"], axis=1)

In [57]:
data_filled.to_csv("data_filled_.tsv", sep='\t')

In [58]:
print("Missing values distribution by column: ")
print(data_filled.isnull().mean())

Missing values distribution by column: 
DOI                       0.000000
Date                      0.083819
Journal                   0.000000
Title                     0.000000
Name                      0.000000
measurement_error         0.000000
measurement_wavelength    0.885628
measurement_method        0.000000
normalised_name           0.129447
raw_value                 0.000000
specifier                 0.000000
dtype: float64


In [27]:
missing_values.to_csv("missing_values_smiles.tsv", sep='\t')

In [30]:
from rdkit import Chem
for index, row in tqdm(data_filled.iterrows()):
    # check if normalised_name is missing a value
    if pd.isna(row["normalised_name"]):
        # generate a SMILES string from the Name column
        mol = Chem.MolFromSmiles(row["Name"])
        if mol is not None:
            smiles = Chem.MolToSmiles(mol)
            # update the normalised_name column with the SMILES string
            data_filled.at[index, "normalised_name"] = smiles

0it [00:00, ?it/s][00:27:10] SMILES Parse Error: syntax error while parsing: Al2O3
[00:27:10] SMILES Parse Error: Failed parsing SMILES 'Al2O3' for input: 'Al2O3'
8it [00:00, 71.20it/s][00:27:10] SMILES Parse Error: syntax error while parsing: Polycarbonate
[00:27:10] SMILES Parse Error: Failed parsing SMILES 'Polycarbonate' for input: 'Polycarbonate'
[00:27:10] SMILES Parse Error: syntax error while parsing: SiO2–ZrO2
[00:27:10] SMILES Parse Error: Failed parsing SMILES 'SiO2–ZrO2' for input: 'SiO2–ZrO2'
[00:27:10] SMILES Parse Error: syntax error while parsing: sapphire
[00:27:10] SMILES Parse Error: Failed parsing SMILES 'sapphire' for input: 'sapphire'
[00:27:10] SMILES Parse Error: syntax error while parsing: TiSi2
[00:27:10] SMILES Parse Error: Failed parsing SMILES 'TiSi2' for input: 'TiSi2'
[00:27:10] SMILES Parse Error: syntax error while parsing: Ca9.3Si10O22.18N4.75
[00:27:10] SMILES Parse Error: Failed parsing SMILES 'Ca9.3Si10O22.18N4.75' for input: 'Ca9.3Si10O22.18N4.75'


[00:27:10] SMILES Parse Error: syntax error while parsing: BiFeO3
[00:27:10] SMILES Parse Error: Failed parsing SMILES 'BiFeO3' for input: 'BiFeO3'
[00:27:10] SMILES Parse Error: syntax error while parsing: WO3-xNd2O3
[00:27:10] SMILES Parse Error: Failed parsing SMILES 'WO3-xNd2O3' for input: 'WO3-xNd2O3'
[00:27:10] SMILES Parse Error: syntax error while parsing: Kaolinite
[00:27:10] SMILES Parse Error: Failed parsing SMILES 'Kaolinite' for input: 'Kaolinite'
[00:27:10] SMILES Parse Error: syntax error while parsing: CH3(CH2)6CH3
[00:27:10] SMILES Parse Error: Failed parsing SMILES 'CH3(CH2)6CH3' for input: 'CH3(CH2)6CH3'
[00:27:10] SMILES Parse Error: syntax error while parsing: SiO2–Al2O3
[00:27:10] SMILES Parse Error: Failed parsing SMILES 'SiO2–Al2O3' for input: 'SiO2–Al2O3'
[00:27:10] SMILES Parse Error: syntax error while parsing: ZnGeAs2
[00:27:10] SMILES Parse Error: Failed parsing SMILES 'ZnGeAs2' for input: 'ZnGeAs2'
[00:27:10] SMILES Parse Error: syntax error while parsing:

[00:27:11] SMILES Parse Error: syntax error while parsing: FeO
[00:27:11] SMILES Parse Error: Failed parsing SMILES 'FeO' for input: 'FeO'
[00:27:11] SMILES Parse Error: syntax error while parsing: Bi2O3–SiO2
[00:27:11] SMILES Parse Error: Failed parsing SMILES 'Bi2O3–SiO2' for input: 'Bi2O3–SiO2'
[00:27:11] SMILES Parse Error: syntax error while parsing: chitin
[00:27:11] SMILES Parse Error: Failed parsing SMILES 'chitin' for input: 'chitin'
[00:27:11] SMILES Parse Error: syntax error while parsing: Ga0.6Al0.4As
[00:27:11] SMILES Parse Error: Failed parsing SMILES 'Ga0.6Al0.4As' for input: 'Ga0.6Al0.4As'
[00:27:11] SMILES Parse Error: syntax error while parsing: OCOCH2CH2
[00:27:11] SMILES Parse Error: Failed parsing SMILES 'OCOCH2CH2' for input: 'OCOCH2CH2'
[00:27:11] SMILES Parse Error: syntax error while parsing: Ge33S67
[00:27:11] SMILES Parse Error: Failed parsing SMILES 'Ge33S67' for input: 'Ge33S67'
[00:27:11] SMILES Parse Error: syntax error while parsing: CdO–B2O3
[00:27:11] 

[00:27:11] SMILES Parse Error: syntax error while parsing: Na2O
[00:27:11] SMILES Parse Error: Failed parsing SMILES 'Na2O' for input: 'Na2O'
[00:27:11] SMILES Parse Error: unclosed ring for input: 'Cs2O–SiO2'
[00:27:11] SMILES Parse Error: syntax error while parsing: Heptene
[00:27:11] SMILES Parse Error: Failed parsing SMILES 'Heptene' for input: 'Heptene'
[00:27:11] SMILES Parse Error: syntax error while parsing: PVC
[00:27:11] SMILES Parse Error: Failed parsing SMILES 'PVC' for input: 'PVC'
[00:27:11] SMILES Parse Error: syntax error while parsing: Al2O3
[00:27:11] SMILES Parse Error: Failed parsing SMILES 'Al2O3' for input: 'Al2O3'
[00:27:11] SMILES Parse Error: syntax error while parsing: PSU
[00:27:11] SMILES Parse Error: Failed parsing SMILES 'PSU' for input: 'PSU'
[00:27:11] SMILES Parse Error: syntax error while parsing: NiTPP
[00:27:11] SMILES Parse Error: Failed parsing SMILES 'NiTPP' for input: 'NiTPP'
[00:27:11] SMILES Parse Error: syntax error while parsing: PPA
[00:27:1

4975it [00:00, 11128.22it/s]


In [14]:
print("Missing values distribution by column: ")
print(data_filled.isnull().mean())

Missing values distribution by column: 
Unnamed: 0                0.000000
DOI                       0.000000
Date                      0.000000
Journal                   0.000000
Title                     0.000000
Name                      0.000000
measurement_error         0.000000
measurement_wavelength    0.885628
measurement_method        0.000000
normalised_name           0.411457
raw_value                 0.000000
specifier                 0.000000
Data                      0.000000
dtype: float64


In [32]:
still_missing_values = data_filled[data_filled["normalised_name"].isnull()]
len(still_missing_values)

640

In [33]:
data_filled = data_filled.drop(["Unnamed: 0", "Data", "SMILES"], axis=1)

In [35]:
still_missing_values

,Unnamed: 0,DOI,Date,Journal,Title,Name,measurement_error,measurement_wavelength,measurement_method,normalised_name,raw_value,specifier,Data,SMILES
7,7,10.1016/j.tsf.2018.01.058,2/2/2018,Thin Solid Films,Density and size effects on the thermal conduc...,Al2O3,0.000,20 nm,el_mylogic,None,1.71,Index of refraction,2/23/2023,None
15,15,10.1016/S0378-7788(00)00124-9,5/9/2001,Energy and Buildings,"Experimental results of transparent, reflectiv...",Polycarbonate,0.000,NaN,el_mylogic,None,1.55,Refraction index,4/5/2022,None
17,17,10.1016/j.optmat.2008.10.005,4/26/2009,Optical Materials,CO2 laser annealing on erbium-activated glass–...,SiO2–ZrO2,0.000,0.3 nm,el_mylogic,None,1.563,Refractive index at 1.3 μm ± 0.005,11/29/2022,None
23,23,10.1039/C3TC30313F,NaN,J. Mater. Chem. C,Mapping the structural and optical properties ...,sapphire,0.000,NaN,snowball,None,1.33,refraction indices,3/31/2022,None
26,26,10.1016/j.physleta.2014.02.028,2/28/2014,Physics Letters A,Comb-like optical transmission spectra generat...,TiSi2,0.000,41.35 μm,el_mylogic,None,1.809,Refractive index,9/29/2022,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4957,4982,10.1016/j.jct.2011.09.011,9/28/2011,The Journal of Chemical Thermodynamics,Measurement and correlation of the excess mola...,C10H21OH,0.000,NaN,el_mylogic,None,1.4374,nD,6/23/2022,None
4958,4983,10.1016/j.jnoncrysol.2018.02.007,2/21/2018,Journal of Non-Crystalline Solids,Calculation of the structure and physical prop...,Na2O·2SiO2,0.000,NaN,el_mylogic,None,1.5072 [],Refractive (nD),3/6/2023,None
4961,4986,10.1016/S0272-8842(01)00005-0,6/12/2001,Ceramics International,The optical waveguide characteristics of highl...,Si ( 111 ),0.000,NaN,el_cde_tables,None,1.78,Refractive index,6/27/2022,None
4962,4987,10.1016/j.ceramint.2018.08.080,8/9/2018,Ceramics International,Structural and spectroscopic properties of Yb3...,BPYb20,0.001,NaN,el_cde_tables,None,1.545 ± 0.001,Refractive index,3/7/2023,None


In [36]:
# define the base URL for the PubChem API
url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/"

# iterate over the rows in the DataFrame
for index, row in tqdm(data_filled.iterrows()):
    # check if the normalised_name is missing a value
    if pd.isna(row["normalised_name"]):
        # get the name of the compound
        name = row["Name"]
        # make a request to the PubChem API to get the compound CID
        cid_url = url + "name/" + name + "/cids/JSON"
        cid_response = requests.get(cid_url)
        cid_data = cid_response.json()
        # check if a CID was returned
        if "IdentifierList" in cid_data and "CID" in cid_data["IdentifierList"][0]:
            cid = cid_data["IdentifierList"][0]["CID"]
            # make a request to the PubChem API to get the SMILES string
            smiles_url = url + "cid/" + str(cid) + "/property/IsomerSMILES/JSON"
            smiles_response = requests.get(smiles_url)
            smiles_data = smiles_response.json()
            # check if a SMILES string was returned
            if "PropertyTable" in smiles_data and "IsomerSMILES" in smiles_data["PropertyTable"][0]:
                smiles = smiles_data["PropertyTable"][0]["IsomerSMILES"]
                # update the normalised_name column with the SMILES string
                data_filled.at[index, "normalised_name"] = smiles

4975it [05:34, 14.87it/s]


In [15]:
print("Missing values distribution by column: ")
print(data_filled.isnull().mean())

Missing values distribution by column: 
Unnamed: 0                0.000000
DOI                       0.000000
Date                      0.000000
Journal                   0.000000
Title                     0.000000
Name                      0.000000
measurement_error         0.000000
measurement_wavelength    0.885628
measurement_method        0.000000
normalised_name           0.411457
raw_value                 0.000000
specifier                 0.000000
Data                      0.000000
dtype: float64


In [25]:
still_still_missing_values = data_filled[data_filled["normalised_name"].isnull()]
still_still_missing_values

,Unnamed: 0,DOI,Date,Journal,Title,Name,measurement_error,measurement_wavelength,measurement_method,normalised_name,raw_value,specifier,Data
3,3,10.1039/B717069F,3/7/2023,J. Mater. Chem.,High refractive index polyimide–nanocrystallin...,BK7,0.00,NaN,snowball,NaN,1.518,refractive index,3/7/2023
7,7,10.1016/j.tsf.2018.01.058,2/2/2018,Thin Solid Films,Density and size effects on the thermal conduc...,Al2O3,0.00,20 nm,el_mylogic,NaN,1.71,Index of refraction,2/23/2023
10,10,10.1016/j.atmosres.2006.02.003,6/2/2006,Atmospheric Research,Observations of supersaturated MgSO4 and NaClO...,ZnSe,0.00,NaN,el_cde_text,NaN,2.4,refractive index,2/28/2023
12,12,10.1016/j.jct.2015.12.025,1/6/2016,The Journal of Chemical Thermodynamics,"The physicochemical properties of 1,3,5-trimet...",nitrobenzene,0.00,NaN,el_cde_tables,NaN,1.38845,nD,6/1/2022
15,15,10.1016/S0378-7788(00)00124-9,5/9/2001,Energy and Buildings,"Experimental results of transparent, reflectiv...",Polycarbonate,0.00,NaN,el_mylogic,NaN,1.55,Refraction index,4/5/2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4965,4990,10.1016/j.optmat.2014.04.044,5/16/2014,Optical Materials,Luminescence properties of silk cocoon derived...,Quinine sulfate,0.00,NaN,el_cde_tables,NaN,1.33,Refractive index,3/6/2023
4966,4991,10.1039/B413474E,1/20/2005,J. Mater. Chem.,A novel organometallic route to phenylethenyl-...,styrene,0.04,NaN,rsc_cde_text,NaN,1.51 to 1.59,refractive index,4/3/2022
4967,4992,10.1016/j.molliq.2015.03.012,3/10/2015,Journal of Molecular Liquids,Quantum mechanical study of solvation analysis...,Toluene,0.00,NaN,el_mylogic,NaN,1.5,n,2/24/2023
4968,4993,10.1016/j.jphotochem.2017.08.050,8/24/2017,Journal of Photochemistry and Photobiology A: ...,Photo-physical behaviors of various active for...,Acetone,0.00,NaN,el_mylogic,NaN,1.359,n,2/24/2023


In [16]:
data_filled.to_csv("data_filled.tsv", sep='\t')

In [41]:
still_still_missing_values.to_csv("still_still_missing_values.tsv", sep='\t')

In [42]:
from chemspipy import ChemSpider

# initialize the ChemSpider API client
cs = ChemSpider("YOUR_API_KEY")

# iterate over the rows in the DataFrame
for index, row in tqdm(data_filled.iterrows()):
    # check if the normalised_name is missing a value
    if pd.isna(row["normalised_name"]):
        # get the name of the compound
        name = row["Name"]
        # search for the compound in ChemSpider
        results = cs.search(name)
        # check if any results were found
        if len(results) > 0:
            # get the SMILES string of the first result
            smiles = results[0].smiles
            # update the normalised_name column with the SMILES string
            data_filled.at[index, "normalised_name"] = smiles

4975it [00:46, 106.52it/s]


In [20]:
print("Missing values distribution by column: ")
print(data_filled.isnull().mean())

Missing values distribution by column: 
Unnamed: 0                0.000000
DOI                       0.000000
Date                      0.000000
Journal                   0.000000
Title                     0.000000
Name                      0.000000
measurement_error         0.000000
measurement_wavelength    0.885628
measurement_method        0.000000
normalised_name           0.411457
raw_value                 0.000000
specifier                 0.000000
Data                      0.000000
dtype: float64


In [23]:
data_filled.to_csv("data_filled.tsv", sep='\t')

In [26]:
still_still_missing_values['good_name'] = pd.Series([None]*len(still_still_missing_values))

C:\Users\bocha\AppData\Local\Temp\ipykernel_3080\1771489550.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  still_still_missing_values['good_name'] = pd.Series([None]*len(still_still_missing_values))


In [27]:
still_still_missing_values

,Unnamed: 0,DOI,Date,Journal,Title,Name,measurement_error,measurement_wavelength,measurement_method,normalised_name,raw_value,specifier,Data,good_name
3,3,10.1039/B717069F,3/7/2023,J. Mater. Chem.,High refractive index polyimide–nanocrystallin...,BK7,0.00,NaN,snowball,NaN,1.518,refractive index,3/7/2023,None
7,7,10.1016/j.tsf.2018.01.058,2/2/2018,Thin Solid Films,Density and size effects on the thermal conduc...,Al2O3,0.00,20 nm,el_mylogic,NaN,1.71,Index of refraction,2/23/2023,None
10,10,10.1016/j.atmosres.2006.02.003,6/2/2006,Atmospheric Research,Observations of supersaturated MgSO4 and NaClO...,ZnSe,0.00,NaN,el_cde_text,NaN,2.4,refractive index,2/28/2023,None
12,12,10.1016/j.jct.2015.12.025,1/6/2016,The Journal of Chemical Thermodynamics,"The physicochemical properties of 1,3,5-trimet...",nitrobenzene,0.00,NaN,el_cde_tables,NaN,1.38845,nD,6/1/2022,None
15,15,10.1016/S0378-7788(00)00124-9,5/9/2001,Energy and Buildings,"Experimental results of transparent, reflectiv...",Polycarbonate,0.00,NaN,el_mylogic,NaN,1.55,Refraction index,4/5/2022,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4965,4990,10.1016/j.optmat.2014.04.044,5/16/2014,Optical Materials,Luminescence properties of silk cocoon derived...,Quinine sulfate,0.00,NaN,el_cde_tables,NaN,1.33,Refractive index,3/6/2023,NaN
4966,4991,10.1039/B413474E,1/20/2005,J. Mater. Chem.,A novel organometallic route to phenylethenyl-...,styrene,0.04,NaN,rsc_cde_text,NaN,1.51 to 1.59,refractive index,4/3/2022,NaN
4967,4992,10.1016/j.molliq.2015.03.012,3/10/2015,Journal of Molecular Liquids,Quantum mechanical study of solvation analysis...,Toluene,0.00,NaN,el_mylogic,NaN,1.5,n,2/24/2023,NaN
4968,4993,10.1016/j.jphotochem.2017.08.050,8/24/2017,Journal of Photochemistry and Photobiology A: ...,Photo-physical behaviors of various active for...,Acetone,0.00,NaN,el_mylogic,NaN,1.359,n,2/24/2023,NaN


In [28]:
import html
# define the base URL for the ChemSpider API
url = "http://www.chemspider.com/"

# iterate over the rows in the DataFrame
for index, row in tqdm(still_still_missing_values.iterrows()):
    # get the name of the compound
    name = row["Name"]
    # make a request to the ChemSpider API to get the systematic name
    r = requests.get(url + "Search.asmx/SimpleSearch?query=" + name)
    if r.status_code == 200:
        # parse the XML response and get the systematic name
        systematic_name = r.content.decode("utf-8")
        systematic_name = systematic_name.split("<SystematicName>")[1].split("</SystematicName>")[0]
        # update the 'good_name' column with the retrieved name
        still_still_missing_values.at[index, "good_name"] = html.unescape(systematic_name)

26it [00:16,  1.56it/s]

KeyboardInterrupt



In [ ]:
still_still_missing_values.to_csv("still_still_missing_values.tsv", sep='\t')

In [ ]:
from chemspipy import ChemSpider

# initialize the ChemSpider API client
cs = ChemSpider("YOUR_API_KEY")

# iterate over the rows in the DataFrame
for index, row in tqdm(still_still_missing_values.iterrows()):
    # check if the normalised_name is missing a value
    if pd.isna(row["normalised_name"]):
        # get the name of the compound
        name = row["good_name"]
        # search for the compound in ChemSpider
        results = cs.search(name)
        # check if any results were found
        if len(results) > 0:
            # get the SMILES string of the first result
            smiles = results[0].smiles
            # update the normalised_name column with the SMILES string
            still_still_missing_values.at[index, "normalised_name"] = smiles

In [62]:
print("Missing values distribution by column: ")
print(data_filled.isnull().mean())

data_date = pd.read_csv('./data_filled_EXTRA.tsv', sep='\t')
data_date = data_date.drop(['Unnamed: 0.1', 'Unnamed: 0', 'Data'], axis=1)
print("Missing values distribution by column: ")
print(data_date.isnull().mean())

Missing values distribution by column: 
DOI                       0.000000
Date                      0.083819
Journal                   0.000000
Title                     0.000000
Name                      0.000000
measurement_error         0.000000
measurement_wavelength    0.885628
measurement_method        0.000000
normalised_name           0.129447
raw_value                 0.000000
specifier                 0.000000
dtype: float64
Missing values distribution by column: 
DOI                       0.000000
Date                      0.000201
Journal                   0.000000
Title                     0.000000
Name                      0.000000
measurement_error         0.000000
measurement_wavelength    0.885628
measurement_method        0.000000
normalised_name           0.411457
raw_value                 0.000000
specifier                 0.000000
dtype: float64


In [63]:
data_filled['Date'] = data_date['Date'].values
print("Missing values distribution by column: ")
print(data_filled.isnull().mean())

Missing values distribution by column: 
DOI                       0.000000
Date                      0.000201
Journal                   0.000000
Title                     0.000000
Name                      0.000000
measurement_error         0.000000
measurement_wavelength    0.885628
measurement_method        0.000000
normalised_name           0.129447
raw_value                 0.000000
specifier                 0.000000
dtype: float64


In [31]:
data_ = pd.read_csv('data_filled.tsv', sep='\t')

In [32]:
print("Missing values distribution by column: ")
print(data_.isnull().mean())

Missing values distribution by column: 
Unnamed: 0.1              0.000000
Unnamed: 0                0.000000
DOI                       0.000000
Date                      0.000201
Journal                   0.000000
Title                     0.000000
Name                      0.000000
measurement_error         0.000000
measurement_wavelength    0.885628
measurement_method        0.000000
normalised_name           0.411457
raw_value                 0.000000
specifier                 0.000000
Data                      0.000000
dtype: float64


In [33]:
data_0 = pd.read_csv('data_filled_EXTRA.tsv', sep='\t')

In [35]:
print("Missing values distribution by column: ")
print(data_0.isnull().mean())

Missing values distribution by column: 
Unnamed: 0.1              0.000000
Unnamed: 0                0.000000
DOI                       0.000000
Date                      0.000201
Journal                   0.000000
Title                     0.000000
Name                      0.000000
measurement_error         0.000000
measurement_wavelength    0.885628
measurement_method        0.000000
normalised_name           0.411457
raw_value                 0.000000
specifier                 0.000000
Data                      0.000000
dtype: float64


In [38]:
data_1 = data_0.drop(["Unnamed: 0.1", "Unnamed: 0"], axis=1)

In [39]:
print("Missing values distribution by column: ")
print(data_1.isnull().mean())

Missing values distribution by column: 
DOI                       0.000000
Date                      0.000201
Journal                   0.000000
Title                     0.000000
Name                      0.000000
measurement_error         0.000000
measurement_wavelength    0.885628
measurement_method        0.000000
normalised_name           0.411457
raw_value                 0.000000
specifier                 0.000000
Data                      0.000000
dtype: float64


In [40]:
data_1

,DOI,Date,Journal,Title,Name,measurement_error,measurement_wavelength,measurement_method,normalised_name,raw_value,specifier,Data
0,10.1016/j.optlaseng.2008.09.005,1/14/2009,Optics and Lasers in Engineering,A high-precision measurement technique for eva...,Alcohol,0.00000,NaN,el_mylogic,CCO,1.34749,Refractive index (n),2/14/2023
1,10.1016/j.jiec.2018.12.038,12/29/2018,Journal of Industrial and Engineering Chemistry,Improving understanding of solvent effects on ...,aceticacid,0.00000,NaN,el_mylogic,CC(O)=O,1.7,n,2/13/2023
2,10.1016/j.jlumin.2015.02.010,2/14/2015,Journal of Luminescence,Spectroscopic properties of tellurite glasses ...,Phosphate,0.00000,600 nm,el_mylogic,[O-][P]([O-])([O-])=O,1.54,Refractive index (~600 nm),12/27/2022
3,10.1039/B717069F,3/7/2023,J. Mater. Chem.,High refractive index polyimide–nanocrystallin...,BK7,0.00000,NaN,snowball,NaN,1.518,refractive index,3/7/2023
4,10.1529/biophysj.106.094946,10/23/2008,Biophysical Journal,Quantitative Characterization of Biological Li...,ethanol,0.00126,NaN,el_cde_text,CCO,1.35434 ± 0.00126,refractive index,3/7/2023
...,...,...,...,...,...,...,...,...,...,...,...,...
4970,10.1016/j.sse.2007.06.004,7/30/2007,Solid-State Electronics,Radiative line width of a single-impurity mole...,Biphenyl,0.00000,NaN,el_cde_tables,c1ccc(cc1)c2ccccc2,1.68,Refractive indices,3/29/2022
4971,10.1039/B822982A,7/21/2009,Lab Chip,Tunable Liquid Gradient Refractive Index (L-GR...,CaCl2,0.00000,NaN,rsc_cde_text,"[['Ca', 1.0], ['Cl', 2.0]]",∼1.41,nD,2/19/2023
4972,10.1078/0030-4026-00175,11/5/2004,Optik,Optimization of conducting polymer thin film o...,He-Ne,0.00000,NaN,el_cde_text,"[['He-', 1.0], ['Ne', 1.0]]",1.717,refractive index,1/27/2023
4973,10.1016/j.optmat.2018.06.013,6/18/2018,Optical Materials,Enhanced photoelectric performance of nanorod ...,ITO,0.00000,NaN,el_cde_text,"[['IT', 1.0], ['O', 1.0]]",1.92,refractive index,11/18/2022


In [41]:
missing_data_rows = data_1.loc[data_1['Date'].isnull()]
print(missing_data_rows)

                           DOI Date                      Journal  \
135  10.1016/j.mee.2004.03.068  NaN  Microelectronic Engineering   

                                                 Title     Name  \
135  On-chip optical components and microfluidic sy...  ethanol   

     measurement_error measurement_wavelength measurement_method  \
135                0.0                    NaN           snowball   

    normalised_name raw_value specifier        Data  
135             NaN      1.36         n  06/22/2004  


In [43]:
data_1.loc[135, 'Date'] = data_1.loc[135, 'Data']

In [44]:
print("Missing values distribution by column: ")
print(data_1.isnull().mean())

Missing values distribution by column: 
DOI                       0.000000
Date                      0.000000
Journal                   0.000000
Title                     0.000000
Name                      0.000000
measurement_error         0.000000
measurement_wavelength    0.885628
measurement_method        0.000000
normalised_name           0.411457
raw_value                 0.000000
specifier                 0.000000
Data                      0.000000
dtype: float64


In [64]:
data_filled.to_csv('data_KONECHNOE.tsv', sep='\t', index=False)

### Fixing raw values

In [66]:
print("Missing values distribution by column: ")
print(data_filled.isnull().mean())

Missing values distribution by column: 
DOI                       0.000000
Date                      0.000201
Journal                   0.000000
Title                     0.000000
Name                      0.000000
measurement_error         0.000000
measurement_wavelength    0.885628
measurement_method        0.000000
normalised_name           0.129447
raw_value                 0.000000
specifier                 0.000000
dtype: float64


In [67]:
raw_value_pattern = r'^\d+(\.\d+)?$'

In [69]:
import re

# Calculate the percentage of values that match the pattern
matches = data_filled['raw_value'].str.match(raw_value_pattern).sum()
total = len(data_filled['raw_value'])
match_percent = 100 * matches / total
print(f"Percentage of values that match the pattern: {match_percent:.2f}%")

Percentage of values that match the pattern: 91.44%


In [70]:
non_matches = data_filled[~data_filled['raw_value'].str.match(raw_value_pattern)]['raw_value']
print("Values that do not match the pattern:")
print(non_matches)

Values that do not match the pattern:
4       1.35434 ± 0.00126
18              2.48±0.02
43                1.66 []
56              1.45–1.46
61             1.9 to 2.7
              ...        
4932          1.14 ± 0.02
4958            1.5072 []
4962        1.545 ± 0.001
4966         1.51 to 1.59
4971                ∼1.41
Name: raw_value, Length: 426, dtype: object


In [94]:
import re
import difflib

# Get the non-matching values
non_matches = data_filled[~data_filled['raw_value'].str.match(pattern)]['raw_value']

# Define a function to find the closest match to a string in a list
def find_closest_match(string, string_list):
    matches = difflib.get_close_matches(string, string_list, n=1, cutoff=0.8)
    if matches:
        return matches[0]
    else:
        return None

# Group the values by pattern similarity
groups = {}
for value in non_matches:
    pattern = re.sub(r'\d+(\.\d+)?', r'X', value)
    match = find_closest_match(pattern, groups.keys())
    if match:
        groups[match].append(value)
    else:
        groups[pattern] = [value]

# Store the patterns and values in separate lists
patterns = []
for pattern, pattern_values in groups.items():
    patterns.append(pattern)

# Print out the groups
for pattern, values in groups.items():
    print(f"Pattern {pattern}:")
    print(values)
    
# Print out the patterns and values
print(f"Patterns: {patterns}")


Pattern X:
['1.34749', '1.7', '1.54', '1.518', '1.344', '1.3045', '1.71', '1.98', '2.55', '2.4', '1.42', '1.38845', '1.4057', '1.432', '1.55', '1.63', '1.563', '3.71', '1.55', '1.4', '1.33', '1.33', '1.4', '1.4315', '1.809', '3.488', '1.359', '1.78', '3.1291', '4.78', '1.7', '1.40107', '1.418', '1.333', '2.56', '1.82', '1.39501', '1.529', '3.86', '4.52', '4.35', '1.765', '1.429', '1.95', '1.55', '1.7', '2.5', '1.05', '1.6125', '2.087', '2.2', '3.3', '1.424', '1.72', '1.62', '1.56', '1.486', '6', '1.44', '1.529', '2.18', '1.43', '1.3776', '2.11', '1.44345', '1.336787', '1.42', '1.458', '1.4', '2.94', '2.392', '2.164', '1.61', '3', '1.3356', '1.4402', '2.12', '1.3878', '1.56', '1.8', '6', '2.6', '1.36', '1.511', '3.1', '2.3', '1.409', '2.562', '1.3614', '1.24', '3.32', '1.406', '1.3265', '1.3615', '1.612', '1.399', '1.697', '1.387', '1.45', '2', '2.05', '1.971', '1.343', '1.5978', '2.958', '1.5289', '1.17', '1.8', '1.52', '2.1', '2.8', '1.5651', '1.355', '1.344', '1.39', '1.3347', '1.531

In [114]:
import re
import difflib

# Define a function to find the closest match to a string in a list
def find_closest_match(string, string_list):
    matches = difflib.get_close_matches(string, string_list, n=1, cutoff=0.8)
    if matches:
        return matches[0]
    else:
        return None

# Define a function to get the pattern for a given value
def get_pattern(value, pattern_groups):
    pattern = re.sub(r'\d+(\.\d+)?', r'X', value)
    match = find_closest_match(pattern, pattern_groups.keys())
    if match:
        return match
    else:
        return pattern

# Group the values by pattern similarity
pattern_groups = {}
for value in data_filled['raw_value']:
    pattern = get_pattern(value, pattern_groups)
    if pattern in pattern_groups:
        pattern_groups[pattern].append(value)
    else:
        pattern_groups[pattern] = [value]

# Create a new column with the patterns
data_filled['patterns'] = data_filled['raw_value'].apply(get_pattern, args=(pattern_groups,))

# Print out the groups
for pattern, values in pattern_groups.items():
    print(f"Pattern {pattern}:")
    print(values)

# Print out the patterns
print(f"Patterns: {list(pattern_groups.keys())}")

Pattern X:
['1.34749', '1.7', '1.54', '1.518', '1.344', '1.3045', '1.71', '1.98', '2.55', '2.4', '1.42', '1.38845', '1.4057', '1.432', '1.55', '1.63', '1.563', '3.71', '1.55', '1.4', '1.33', '1.33', '1.4', '1.4315', '1.809', '3.488', '1.359', '1.78', '3.1291', '4.78', '1.7', '1.40107', '1.418', '1.333', '2.56', '1.82', '1.39501', '1.529', '3.86', '4.52', '4.35', '1.765', '1.429', '1.95', '1.55', '1.7', '2.5', '1.05', '1.6125', '2.087', '2.2', '3.3', '1.424', '1.72', '1.62', '1.56', '1.486', '6', '1.44', '1.529', '2.18', '1.43', '1.3776', '2.11', '1.44345', '1.336787', '1.42', '1.458', '1.4', '2.94', '2.392', '2.164', '1.61', '3', '1.3356', '1.4402', '2.12', '1.3878', '1.56', '1.8', '6', '2.6', '1.36', '1.511', '3.1', '2.3', '1.409', '2.562', '1.3614', '1.24', '3.32', '1.406', '1.3265', '1.3615', '1.612', '1.399', '1.697', '1.387', '1.45', '2', '2.05', '1.971', '1.343', '1.5978', '2.958', '1.5289', '1.17', '1.8', '1.52', '2.1', '2.8', '1.5651', '1.355', '1.344', '1.39', '1.3347', '1.531

In [115]:
data_filled

,DOI,Date,Journal,Title,Name,measurement_error,measurement_wavelength,measurement_method,normalised_name,raw_value,specifier,patterns
0,10.1016/j.optlaseng.2008.09.005,1/14/2009,Optics and Lasers in Engineering,A high-precision measurement technique for eva...,Alcohol,0.00000,NaN,el_mylogic,CCO,1.34749,Refractive index (n),X
1,10.1016/j.jiec.2018.12.038,12/29/2018,Journal of Industrial and Engineering Chemistry,Improving understanding of solvent effects on ...,aceticacid,0.00000,NaN,el_mylogic,CC(O)=O,1.7,n,X
2,10.1016/j.jlumin.2015.02.010,2/14/2015,Journal of Luminescence,Spectroscopic properties of tellurite glasses ...,Phosphate,0.00000,600 nm,el_mylogic,[O-][P]([O-])([O-])=O,1.54,Refractive index (~600 nm),X
3,10.1039/B717069F,3/7/2023,J. Mater. Chem.,High refractive index polyimide–nanocrystallin...,BK7,0.00000,NaN,snowball,CCOC1=CC2=C(C=C1)C=C(C=C2)C3=NN(C4=NC=NC(=C34)...,1.518,refractive index,X
4,10.1529/biophysj.106.094946,10/23/2008,Biophysical Journal,Quantitative Characterization of Biological Li...,ethanol,0.00126,NaN,el_cde_text,CCO,1.35434 ± 0.00126,refractive index,X ± X
...,...,...,...,...,...,...,...,...,...,...,...,...
4970,10.1016/j.sse.2007.06.004,7/30/2007,Solid-State Electronics,Radiative line width of a single-impurity mole...,Biphenyl,0.00000,NaN,el_cde_tables,c1ccc(cc1)c2ccccc2,1.68,Refractive indices,X
4971,10.1039/B822982A,7/21/2009,Lab Chip,Tunable Liquid Gradient Refractive Index (L-GR...,CaCl2,0.00000,NaN,rsc_cde_text,"[['Ca', 1.0], ['Cl', 2.0]]",∼1.41,nD,∼X
4972,10.1078/0030-4026-00175,11/5/2004,Optik,Optimization of conducting polymer thin film o...,He-Ne,0.00000,NaN,el_cde_text,"[['He-', 1.0], ['Ne', 1.0]]",1.717,refractive index,X
4973,10.1016/j.optmat.2018.06.013,6/18/2018,Optical Materials,Enhanced photoelectric performance of nanorod ...,ITO,0.00000,NaN,el_cde_text,"[['IT', 1.0], ['O', 1.0]]",1.92,refractive index,X


As the result i have a list of patterns:
['X', 'X±X', 'X []', 'X–X', 'X to X', 'X−X', 'X ± X', 'X ± X', 'X (X)', '∼X', '~X', 'X ± X', 'X-X', 'X, X, X', 'X + iX', 'X (EtOH)', 'X, X', 'X +', 'X,X ± X,X', 'X at X μm', '<X', 'X–X (this)', 'X X X X', 'X*', 'X+Xi', 'X; X; X', 'Xe']
I want you to write a code for each pattern:
- If the pattern is the 'X', remains the original value
- if the pattern is the 'X±X', remains the first X, the X after '±' deleats
- if the pattern is the 'X []', remains the X, the ' []' delleats
- if the pattern is the 'X–X', the mean of two X before and after '–' should be found
- if the pattern is the 'X to X', the the mean of two X before and after ' to ' should be found
- if the pattern is the 'X−X', the mean of two X before and after '−' should be found
- if the pattern is the 'X ± X', remains the first X, the X after ' ± ' should be deleated
- if the pattern is the 'X ± X', remains the first X, the X after ' ± ' should be deleated
- if the pattern is the 'X (X)', the '(' and ')' should be deleated
- if the pattern is the '∼X', remains the X, the '∼' should be deleated
- if the pattern is the '~X', remains the X, the '~' should be deleated
- if the pattern is the 'X ± X', remains the first X, the X after ' ± ' should be deleated
- if the pattern is the 'X-X', the mean of two X before and after '-' should be found
- if the pattern is the 'X, X, X', the mean of three X should be found
- if the pattern is the 'X + iX', remains the first X, the X after ' + ' should be deleated
- if the pattern is the 'X (EtOH)', remain the X, ' (EtOH)' should be deleated
- if the pattern is the 'X, X', the mean of two X before and after ', ' should be found
- if the pattern is the 'X +', remain the X, ' +' should be deleated
- if the pattern is the 'X,X ± X,X', the ',' should be replaced with '.', and then everything after '±' should be deleated
- if the pattern is the 'X at X μm', remain the first X, ' at X μm' should be deleated
- if the pattern is the '<X', remain the X, '<' should be deleated
- if the pattern is the 'X–X (this)', the the mean of two X before and after '–' should be found, ' (this)' should be deleated
- if the pattern is the 'X X X X', the the mean of four X should be found
- if the pattern is the 'X*', remain the X, '*' should be deleated
- if the pattern is the 'X+Xi', the the mean of two X before and after '+' should be found, 'i' should be deleated
- if the pattern is the 'X; X; X', the the mean of three X should be found
- if the pattern is the 'Xe', remain the X, 'e' should be deleated

In [116]:
def process_value(value, pattern):
    if not value:
        return None
    if pattern == 'X':
        return value
    elif pattern == 'X±X':
        return value.split('±')[0]
    elif pattern == 'X []':
        return value.split(' [')[0]
    elif pattern == 'X–X' or pattern == 'X to X' or pattern == 'X−X' or pattern == 'X-X':
        values = re.findall('\d+(\.\d+)?', value)
        return str(np.mean([float(v) for v in values if v.strip() != '']))
    elif pattern == 'X ± X' or pattern == 'X ± X':
        return value.split('±')[0].split('±')[0]
    elif pattern == 'X (X)':
        return value.split('(')[0].split(')')[0]
    elif pattern == '∼X' or pattern == '~X':
        return value.split('∼')[0].split('~')[0]
    elif pattern == 'X, X, X' or pattern == 'X, X' or pattern == 'X,X ± X,X':
        values = re.findall('\d+(\.\d+)?', value)
        return str(np.mean([float(v) for v in values if v.strip() != '']))
    elif pattern == 'X + iX' or pattern == 'X+Xi':
        return value.split(' +')[0]
    elif pattern == 'X (EtOH)':
        return value.split(' (')[0]
    elif pattern == 'X at X μm':
        return value.split(' at ')[0]
    elif '<' in value and pattern == '<X':
        return float(value.split('<')[1])
    elif pattern == 'X–X (this)':
        values = re.findall('\d+(\.\d+)?', value)
        return str(np.mean([float(v) for v in values if v.strip() != '']))
    elif pattern == 'X X X X':
        values = re.findall('\d+(\.\d+)?', value)
        return str(np.mean([float(v) for v in values if v.strip() != '']))
    elif pattern == 'X*':
        return value.split('*')[0]
    elif pattern == 'X +':
        return value.split(' +')[0]
    elif pattern == 'X; X; X':
        values = re.findall('\d+(\.\d+)?', value)
        return str(np.mean([float(v) for v in values if v.strip() != '']))
    elif pattern == 'Xe':
        return value.split('e')[0]
    else:
        return

In [117]:
patterns = list(data_filled['patterns'])

In [135]:
values = list(data_filled['raw_value'])
import pandas as pd

# create a DataFrame from the list
df = pd.DataFrame({'values': values})

# save the DataFrame to an Excel file
df.to_excel('values.xlsx', index=False)

In [119]:
process_value(values, patterns)

In [120]:
# assuming data_filled and patterns are already defined
clean_values = [process_value(value, pattern) for value, pattern in zip(data_filled['raw_value'], patterns)]

C:\Users\bocha\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\bocha\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [121]:
len(clean_values)

4975

In [136]:
clean_values = pd.read_excel('./values.xlsx')

In [140]:
print(len(data_filled))
print(len(clean_values))

4975
4975


In [145]:
data_filled['raw_value'] = clean_values['values']

In [146]:
print("Missing values distribution by column: ")
print(data_filled.isnull().mean())

Missing values distribution by column: 
DOI                       0.000000
Date                      0.000201
Journal                   0.000000
Title                     0.000000
Name                      0.000000
measurement_error         0.000000
measurement_wavelength    0.885628
measurement_method        0.000000
normalised_name           0.129447
raw_value                 0.000000
specifier                 0.000000
patterns                  0.000000
raw_values                0.000000
dtype: float64


In [147]:
data_filled

,DOI,Date,Journal,Title,Name,measurement_error,measurement_wavelength,measurement_method,normalised_name,raw_value,specifier,patterns,raw_values
0,10.1016/j.optlaseng.2008.09.005,1/14/2009,Optics and Lasers in Engineering,A high-precision measurement technique for eva...,Alcohol,0.00000,NaN,el_mylogic,CCO,1.34749,Refractive index (n),X,1.34749
1,10.1016/j.jiec.2018.12.038,12/29/2018,Journal of Industrial and Engineering Chemistry,Improving understanding of solvent effects on ...,aceticacid,0.00000,NaN,el_mylogic,CC(O)=O,1.70000,n,X,1.70000
2,10.1016/j.jlumin.2015.02.010,2/14/2015,Journal of Luminescence,Spectroscopic properties of tellurite glasses ...,Phosphate,0.00000,600 nm,el_mylogic,[O-][P]([O-])([O-])=O,1.54000,Refractive index (~600 nm),X,1.54000
3,10.1039/B717069F,3/7/2023,J. Mater. Chem.,High refractive index polyimide–nanocrystallin...,BK7,0.00000,NaN,snowball,CCOC1=CC2=C(C=C1)C=C(C=C2)C3=NN(C4=NC=NC(=C34)...,1.51800,refractive index,X,1.51800
4,10.1529/biophysj.106.094946,10/23/2008,Biophysical Journal,Quantitative Characterization of Biological Li...,ethanol,0.00126,NaN,el_cde_text,CCO,1.35434,refractive index,X ± X,1.35434
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4970,10.1016/j.sse.2007.06.004,7/30/2007,Solid-State Electronics,Radiative line width of a single-impurity mole...,Biphenyl,0.00000,NaN,el_cde_tables,c1ccc(cc1)c2ccccc2,1.68000,Refractive indices,X,1.68000
4971,10.1039/B822982A,7/21/2009,Lab Chip,Tunable Liquid Gradient Refractive Index (L-GR...,CaCl2,0.00000,NaN,rsc_cde_text,"[['Ca', 1.0], ['Cl', 2.0]]",1.41000,nD,∼X,1.41000
4972,10.1078/0030-4026-00175,11/5/2004,Optik,Optimization of conducting polymer thin film o...,He-Ne,0.00000,NaN,el_cde_text,"[['He-', 1.0], ['Ne', 1.0]]",1.71700,refractive index,X,1.71700
4973,10.1016/j.optmat.2018.06.013,6/18/2018,Optical Materials,Enhanced photoelectric performance of nanorod ...,ITO,0.00000,NaN,el_cde_text,"[['IT', 1.0], ['O', 1.0]]",1.92000,refractive index,X,1.92000


In [150]:
data_filled = data_filled.drop(['patterns', 'raw_values'], axis=1)

In [151]:
data_filled

,DOI,Date,Journal,Title,Name,measurement_error,measurement_wavelength,measurement_method,normalised_name,raw_value,specifier
0,10.1016/j.optlaseng.2008.09.005,1/14/2009,Optics and Lasers in Engineering,A high-precision measurement technique for eva...,Alcohol,0.00000,NaN,el_mylogic,CCO,1.34749,Refractive index (n)
1,10.1016/j.jiec.2018.12.038,12/29/2018,Journal of Industrial and Engineering Chemistry,Improving understanding of solvent effects on ...,aceticacid,0.00000,NaN,el_mylogic,CC(O)=O,1.70000,n
2,10.1016/j.jlumin.2015.02.010,2/14/2015,Journal of Luminescence,Spectroscopic properties of tellurite glasses ...,Phosphate,0.00000,600 nm,el_mylogic,[O-][P]([O-])([O-])=O,1.54000,Refractive index (~600 nm)
3,10.1039/B717069F,3/7/2023,J. Mater. Chem.,High refractive index polyimide–nanocrystallin...,BK7,0.00000,NaN,snowball,CCOC1=CC2=C(C=C1)C=C(C=C2)C3=NN(C4=NC=NC(=C34)...,1.51800,refractive index
4,10.1529/biophysj.106.094946,10/23/2008,Biophysical Journal,Quantitative Characterization of Biological Li...,ethanol,0.00126,NaN,el_cde_text,CCO,1.35434,refractive index
...,...,...,...,...,...,...,...,...,...,...,...
4970,10.1016/j.sse.2007.06.004,7/30/2007,Solid-State Electronics,Radiative line width of a single-impurity mole...,Biphenyl,0.00000,NaN,el_cde_tables,c1ccc(cc1)c2ccccc2,1.68000,Refractive indices
4971,10.1039/B822982A,7/21/2009,Lab Chip,Tunable Liquid Gradient Refractive Index (L-GR...,CaCl2,0.00000,NaN,rsc_cde_text,"[['Ca', 1.0], ['Cl', 2.0]]",1.41000,nD
4972,10.1078/0030-4026-00175,11/5/2004,Optik,Optimization of conducting polymer thin film o...,He-Ne,0.00000,NaN,el_cde_text,"[['He-', 1.0], ['Ne', 1.0]]",1.71700,refractive index
4973,10.1016/j.optmat.2018.06.013,6/18/2018,Optical Materials,Enhanced photoelectric performance of nanorod ...,ITO,0.00000,NaN,el_cde_text,"[['IT', 1.0], ['O', 1.0]]",1.92000,refractive index


In [152]:
data_filled.to_csv('data_final.tsv', sep='\t', index=False)

### Fixing names

In [153]:
data = pd.read_csv('./data_final.tsv', sep='\t')

In [154]:
data

,DOI,Date,Journal,Title,Name,measurement_error,measurement_wavelength,measurement_method,normalised_name,raw_value,specifier
0,10.1016/j.optlaseng.2008.09.005,1/14/2009,Optics and Lasers in Engineering,A high-precision measurement technique for eva...,Alcohol,0.00000,NaN,el_mylogic,CCO,1.34749,Refractive index (n)
1,10.1016/j.jiec.2018.12.038,12/29/2018,Journal of Industrial and Engineering Chemistry,Improving understanding of solvent effects on ...,aceticacid,0.00000,NaN,el_mylogic,CC(O)=O,1.70000,n
2,10.1016/j.jlumin.2015.02.010,2/14/2015,Journal of Luminescence,Spectroscopic properties of tellurite glasses ...,Phosphate,0.00000,600 nm,el_mylogic,[O-][P]([O-])([O-])=O,1.54000,Refractive index (~600 nm)
3,10.1039/B717069F,3/7/2023,J. Mater. Chem.,High refractive index polyimide–nanocrystallin...,BK7,0.00000,NaN,snowball,CCOC1=CC2=C(C=C1)C=C(C=C2)C3=NN(C4=NC=NC(=C34)...,1.51800,refractive index
4,10.1529/biophysj.106.094946,10/23/2008,Biophysical Journal,Quantitative Characterization of Biological Li...,ethanol,0.00126,NaN,el_cde_text,CCO,1.35434,refractive index
...,...,...,...,...,...,...,...,...,...,...,...
4970,10.1016/j.sse.2007.06.004,7/30/2007,Solid-State Electronics,Radiative line width of a single-impurity mole...,Biphenyl,0.00000,NaN,el_cde_tables,c1ccc(cc1)c2ccccc2,1.68000,Refractive indices
4971,10.1039/B822982A,7/21/2009,Lab Chip,Tunable Liquid Gradient Refractive Index (L-GR...,CaCl2,0.00000,NaN,rsc_cde_text,"[['Ca', 1.0], ['Cl', 2.0]]",1.41000,nD
4972,10.1078/0030-4026-00175,11/5/2004,Optik,Optimization of conducting polymer thin film o...,He-Ne,0.00000,NaN,el_cde_text,"[['He-', 1.0], ['Ne', 1.0]]",1.71700,refractive index
4973,10.1016/j.optmat.2018.06.013,6/18/2018,Optical Materials,Enhanced photoelectric performance of nanorod ...,ITO,0.00000,NaN,el_cde_text,"[['IT', 1.0], ['O', 1.0]]",1.92000,refractive index


In [156]:
import pandas as pd
import pubchempy as pcp

# create a function to get the formula from name using PubChemPy library
def get_formula(name):
    try:
        compound = pcp.get_compounds(name, 'name')
        formula = compound[0].molecular_formula
        return formula
    except:
        return "NaN"

# create a new column 'Formula'
data['Formula'] = ""

# iterate through each row in 'Name' column and get corresponding formula
for i, row in tqdm(data.iterrows()):
    name = row['Name']
    formula = ""
    if name == "NaN": # if name is NaN, set formula to NaN
        formula = "NaN"
    elif name.isnumeric(): # if name is numeric, set formula to name
        formula = name
    else: # if name is a chemical name, get formula using the function
        formula = get_formula(name)
    # set the 'Formula' column for this row to the calculated formula
    data.at[i, 'Formula'] = formula

4975it [52:39,  1.57it/s]


In [157]:
data

,DOI,Date,Journal,Title,Name,measurement_error,measurement_wavelength,measurement_method,normalised_name,raw_value,specifier,Formula
0,10.1016/j.optlaseng.2008.09.005,1/14/2009,Optics and Lasers in Engineering,A high-precision measurement technique for eva...,Alcohol,0.00000,NaN,el_mylogic,CCO,1.34749,Refractive index (n),C2H6O
1,10.1016/j.jiec.2018.12.038,12/29/2018,Journal of Industrial and Engineering Chemistry,Improving understanding of solvent effects on ...,aceticacid,0.00000,NaN,el_mylogic,CC(O)=O,1.70000,n,NaN
2,10.1016/j.jlumin.2015.02.010,2/14/2015,Journal of Luminescence,Spectroscopic properties of tellurite glasses ...,Phosphate,0.00000,600 nm,el_mylogic,[O-][P]([O-])([O-])=O,1.54000,Refractive index (~600 nm),O4P-3
3,10.1039/B717069F,3/7/2023,J. Mater. Chem.,High refractive index polyimide–nanocrystallin...,BK7,0.00000,NaN,snowball,CCOC1=CC2=C(C=C1)C=C(C=C2)C3=NN(C4=NC=NC(=C34)...,1.51800,refractive index,C23H26N6O
4,10.1529/biophysj.106.094946,10/23/2008,Biophysical Journal,Quantitative Characterization of Biological Li...,ethanol,0.00126,NaN,el_cde_text,CCO,1.35434,refractive index,C2H6O
...,...,...,...,...,...,...,...,...,...,...,...,...
4970,10.1016/j.sse.2007.06.004,7/30/2007,Solid-State Electronics,Radiative line width of a single-impurity mole...,Biphenyl,0.00000,NaN,el_cde_tables,c1ccc(cc1)c2ccccc2,1.68000,Refractive indices,C12H10
4971,10.1039/B822982A,7/21/2009,Lab Chip,Tunable Liquid Gradient Refractive Index (L-GR...,CaCl2,0.00000,NaN,rsc_cde_text,"[['Ca', 1.0], ['Cl', 2.0]]",1.41000,nD,CaCl2
4972,10.1078/0030-4026-00175,11/5/2004,Optik,Optimization of conducting polymer thin film o...,He-Ne,0.00000,NaN,el_cde_text,"[['He-', 1.0], ['Ne', 1.0]]",1.71700,refractive index,NaN
4973,10.1016/j.optmat.2018.06.013,6/18/2018,Optical Materials,Enhanced photoelectric performance of nanorod ...,ITO,0.00000,NaN,el_cde_text,"[['IT', 1.0], ['O', 1.0]]",1.92000,refractive index,NaN


In [165]:
print("Missing values distribution by column: ")
print(data.isnull().mean())

Missing values distribution by column: 
DOI                       0.000000
Date                      0.000201
Journal                   0.000000
Title                     0.000000
Name                      0.000000
measurement_error         0.000000
measurement_wavelength    0.885628
measurement_method        0.000000
normalised_name           0.129447
raw_value                 0.000000
specifier                 0.000000
Formula                   0.311558
dtype: float64


In [166]:
data.to_excel('data.xlsx', index=False)

In [167]:
nan_date_rows = data[data['Date'].isna()]
print(nan_date_rows)

                           DOI Date                      Journal  \
135  10.1016/j.mee.2004.03.068  NaN  Microelectronic Engineering   

                                                 Title     Name  \
135  On-chip optical components and microfluidic sy...  ethanol   

     measurement_error measurement_wavelength measurement_method  \
135                0.0                    NaN           snowball   

    normalised_name  raw_value specifier Formula  
135             CCO       1.36         n   C2H6O  


In [168]:
data.loc[135, 'Date'] = '4/15/2004'

In [169]:
print("Missing values distribution by column: ")
print(data.isnull().mean())

Missing values distribution by column: 
DOI                       0.000000
Date                      0.000000
Journal                   0.000000
Title                     0.000000
Name                      0.000000
measurement_error         0.000000
measurement_wavelength    0.885628
measurement_method        0.000000
normalised_name           0.129447
raw_value                 0.000000
specifier                 0.000000
Formula                   0.311558
dtype: float64


In [170]:
data.to_excel('data.xlsx', index=False)

## Getting descriptors